In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
pip install torch transformers datasets matplotlib pandas tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: n

In [4]:
# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from transformers import (
#     GPT2LMHeadModel, 
#     GPT2Tokenizer, 
#     TrainingArguments, 
#     Trainer,
#     DataCollatorForLanguageModeling
# )
# from datasets import load_dataset
# import numpy as np
# from tqdm import tqdm
# import matplotlib.pyplot as plt
# import pandas as pd
# from typing import Dict, List, Tuple
# import math

# class WikiTextDataset(Dataset):
#     """Custom dataset for WikiText-2 with proper tokenization"""
    
#     def __init__(self, texts: List[str], tokenizer, max_length: int = 512):
#         self.tokenizer = tokenizer
#         self.max_length = max_length
#         self.examples = []
        
#         print("Processing texts...")
#         for text in tqdm(texts):
#             if len(text.strip()) > 0:  # Skip empty texts
#                 # Tokenize and create sliding windows
#                 tokens = tokenizer.encode(text, add_special_tokens=True)
                
#                 # Create overlapping sequences
#                 for i in range(0, len(tokens) - 1, max_length // 2):
#                     chunk = tokens[i:i + max_length]
#                     if len(chunk) > 1:  # Need at least 2 tokens (input + target)
#                         self.examples.append(chunk)
    
#     def __len__(self):
#         return len(self.examples)
    
#     def __getitem__(self, idx):
#         return {
#             'input_ids': torch.tensor(self.examples[idx], dtype=torch.long),
#             'attention_mask': torch.ones(len(self.examples[idx]), dtype=torch.long)
#         }

# class NWPEvaluator:
#     """Evaluator for Next Word Prediction metrics"""
    
#     def __init__(self, model, tokenizer, device):
#         self.model = model
#         self.tokenizer = tokenizer
#         self.device = device
#         self.model.eval()
    
#     def calculate_perplexity(self, dataloader) -> float:
#         """Calculate perplexity on evaluation dataset"""
#         total_loss = 0
#         total_tokens = 0
        
#         with torch.no_grad():
#             for batch in tqdm(dataloader, desc="Calculating perplexity"):
#                 input_ids = batch['input_ids'].to(self.device)
#                 attention_mask = batch['attention_mask'].to(self.device)
                
#                 # Shift labels for next token prediction
#                 labels = input_ids.clone()
#                 labels[:, :-1] = input_ids[:, 1:]
#                 labels[:, -1] = -100  # Ignore last token
                
#                 outputs = self.model(
#                     input_ids=input_ids,
#                     attention_mask=attention_mask,
#                     labels=labels
#                 )
                
#                 loss = outputs.loss
#                 # Count valid tokens (not -100)
#                 valid_tokens = (labels != -100).sum().item()
                
#                 total_loss += loss.item() * valid_tokens
#                 total_tokens += valid_tokens
        
#         avg_loss = total_loss / total_tokens
#         perplexity = math.exp(avg_loss)
#         return perplexity
    
#     def calculate_top_k_accuracy(self, dataloader, k_values: List[int] = [1, 5, 10]) -> Dict[int, float]:
#         """Calculate top-k accuracy for next word prediction"""
#         correct_predictions = {k: 0 for k in k_values}
#         total_predictions = 0
        
#         with torch.no_grad():
#             for batch in tqdm(dataloader, desc="Calculating top-k accuracy"):
#                 input_ids = batch['input_ids'].to(self.device)
#                 attention_mask = batch['attention_mask'].to(self.device)
                
#                 for seq_idx in range(input_ids.size(0)):
#                     sequence = input_ids[seq_idx]
#                     mask = attention_mask[seq_idx]
                    
#                     # Get valid length
#                     valid_length = mask.sum().item()
                    
#                     # Predict each position (except last)
#                     for pos in range(1, valid_length):
#                         context = sequence[:pos].unsqueeze(0)
#                         target = sequence[pos].item()
                        
#                         outputs = self.model(context)
#                         logits = outputs.logits[0, -1, :]  # Last position logits
                        
#                         # Get top-k predictions
#                         top_k_max = max(k_values)
#                         top_k_tokens = torch.topk(logits, top_k_max).indices
                        
#                         # Check accuracy for different k values
#                         for k in k_values:
#                             if target in top_k_tokens[:k]:
#                                 correct_predictions[k] += 1
                        
#                         total_predictions += 1
        
#         # Calculate accuracies
#         accuracies = {k: correct_predictions[k] / total_predictions for k in k_values}
#         return accuracies
    
#     def sample_predictions(self, text: str, num_predictions: int = 5) -> List[str]:
#         """Generate sample predictions for demonstration"""
#         tokens = self.tokenizer.encode(text, return_tensors='pt').to(self.device)
        
#         predictions = []
#         with torch.no_grad():
#             for _ in range(num_predictions):
#                 outputs = self.model(tokens)
#                 logits = outputs.logits[0, -1, :]
                
#                 # Sample from distribution
#                 probs = torch.softmax(logits, dim=-1)
#                 next_token = torch.multinomial(probs, 1)
                
#                 predicted_word = self.tokenizer.decode(next_token.item())
#                 predictions.append(predicted_word)
                
#                 # Add to context for next prediction
#                 tokens = torch.cat([tokens, next_token.unsqueeze(0)], dim=1)
        
#         return predictions

# def load_and_prepare_data():
#     """Load WikiText-2 dataset and prepare for training"""
#     print("Loading WikiText-2 dataset...")
#     dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
    
#     # Filter out empty texts
#     train_texts = [text for text in dataset['train']['text'] if len(text.strip()) > 0]
#     valid_texts = [text for text in dataset['validation']['text'] if len(text.strip()) > 0]
#     test_texts = [text for text in dataset['test']['text'] if len(text.strip()) > 0]
    
#     print(f"Train texts: {len(train_texts)}")
#     print(f"Validation texts: {len(valid_texts)}")
#     print(f"Test texts: {len(test_texts)}")
    
#     return train_texts, valid_texts, test_texts

# def setup_model_and_tokenizer():
#     """Initialize GPT-2 model and tokenizer"""
#     print("Setting up GPT-2 model and tokenizer...")
#     model_name = "gpt2"
    
#     tokenizer = GPT2Tokenizer.from_pretrained(model_name)
#     model = GPT2LMHeadModel.from_pretrained(model_name)
    
#     # Add padding token
#     tokenizer.pad_token = tokenizer.eos_token
    
#     return model, tokenizer

# def fine_tune_model(model, tokenizer, train_dataset, eval_dataset, output_dir="./gpt2-wikitext2"):
#     """Fine-tune GPT-2 on WikiText-2"""
    
#     # Data collator for language modeling
#     data_collator = DataCollatorForLanguageModeling(
#         tokenizer=tokenizer,
#         mlm=False,  # GPT-2 is autoregressive, not masked LM
#         pad_to_multiple_of=8,
#         return_tensors="pt"
#     )
    
#     # Training arguments
#     training_args = TrainingArguments(
#         output_dir=output_dir,
#         overwrite_output_dir=True,
#         num_train_epochs=3,
#         per_device_train_batch_size=4,
#         per_device_eval_batch_size=4,
#         warmup_steps=500,
#         logging_steps=100,
#         save_steps=1000,
#         eval_steps=500,
#         eval_strategy="steps",  # Changed from evaluation_strategy
#         load_best_model_at_end=True,
#         save_total_limit=2,
#         gradient_accumulation_steps=2,
#         fp16=True,  # Mixed precision training
#         dataloader_pin_memory=True,
#         learning_rate=5e-5,
#         weight_decay=0.01,
#         adam_epsilon=1e-8,
#         max_grad_norm=1.0,
#         lr_scheduler_type="linear",
#         report_to=[],  # Changed from None to empty list
#     )
    
#     # Initialize trainer
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         data_collator=data_collator,
#         train_dataset=train_dataset,
#         eval_dataset=eval_dataset,
#         tokenizer=tokenizer,
#     )
    
#     # Train the model
#     print("Starting fine-tuning...")
#     trainer.train()
    
#     # Save the fine-tuned model
#     trainer.save_model()
#     tokenizer.save_pretrained(output_dir)
    
#     return trainer

# def evaluate_model(model, tokenizer, test_dataset, device):
#     """Comprehensive evaluation of the fine-tuned model"""
#     print("Evaluating fine-tuned model...")
    
#     # Create test dataloader
#     test_dataloader = DataLoader(
#         test_dataset, 
#         batch_size=8, 
#         shuffle=False,
#         collate_fn=lambda x: {
#             'input_ids': torch.nn.utils.rnn.pad_sequence(
#                 [item['input_ids'] for item in x], 
#                 batch_first=True, 
#                 padding_value=tokenizer.pad_token_id
#             ),
#             'attention_mask': torch.nn.utils.rnn.pad_sequence(
#                 [item['attention_mask'] for item in x], 
#                 batch_first=True, 
#                 padding_value=0
#             )
#         }
#     )
    
#     # Initialize evaluator
#     evaluator = NWPEvaluator(model, tokenizer, device)
    
#     # Calculate perplexity
#     perplexity = evaluator.calculate_perplexity(test_dataloader)
#     print(f"Perplexity: {perplexity:.2f}")
    
#     # Calculate top-k accuracy
#     accuracies = evaluator.calculate_top_k_accuracy(test_dataloader, k_values=[1, 5, 10])
#     print("Top-k Accuracies:")
#     for k, acc in accuracies.items():
#         print(f"  Top-{k}: {acc:.4f}")
    
#     return perplexity, accuracies, evaluator

# def demonstrate_predictions(evaluator, tokenizer):
#     """Demonstrate model predictions on sample texts"""
#     print("\nSample Predictions:")
#     print("=" * 50)
    
#     sample_texts = [
#         "Albert Einstein was a",
#         "The capital of France is",
#         "Machine learning is the",
#         "In the year 2024, artificial intelligence",
#         "The quick brown fox"
#     ]
    
#     for text in sample_texts:
#         predictions = evaluator.sample_predictions(text, num_predictions=3)
#         print(f"Input: '{text}'")
#         print(f"Predictions: {predictions}")
#         print("-" * 30)

# def main():
#     # Set device
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"Using device: {device}")
    
#     # Load data
#     train_texts, valid_texts, test_texts = load_and_prepare_data()
    
#     # Setup model and tokenizer
#     model, tokenizer = setup_model_and_tokenizer()
#     model.to(device)
    
#     # Create datasets
#     print("Creating datasets...")
#     train_dataset = WikiTextDataset(train_texts[:1000], tokenizer)  # Subset for demo
#     eval_dataset = WikiTextDataset(valid_texts[:200], tokenizer)
#     test_dataset = WikiTextDataset(test_texts[:200], tokenizer)
    
#     # Fine-tune model
#     trainer = fine_tune_model(model, tokenizer, train_dataset, eval_dataset)
    
#     # Evaluate model
#     perplexity, accuracies, evaluator = evaluate_model(model, tokenizer, test_dataset, device)
    
#     # Demonstrate predictions
#     demonstrate_predictions(evaluator, tokenizer)
    
#     # Create results summary
#     results = {
#         'Metric': ['Perplexity', 'Top-1 Accuracy', 'Top-5 Accuracy', 'Top-10 Accuracy'],
#         'Value': [f"{perplexity:.2f}", f"{accuracies[1]:.4f}", f"{accuracies[5]:.4f}", f"{accuracies[10]:.4f}"]
#     }
    
#     results_df = pd.DataFrame(results)
#     print("\nFinal Results Summary:")
#     print(results_df.to_string(index=False))
    
#     return model, tokenizer, results_df

# if __name__ == "__main__":
#     # Note: This is a comprehensive implementation
#     # For actual execution, you may want to run sections separately
#     # due to computational requirements
    
#     print("GPT-2 Fine-tuning Pipeline for Next Word Prediction")
#     print("=" * 60)
    
#     # Uncomment the following line to run the full pipeline
#     # model, tokenizer, results = main()
    
#     print("Pipeline setup complete!")
#     print("To run: uncomment the main() call and execute with sufficient GPU memory")

GPT-2 Fine-tuning Pipeline for Next Word Prediction
Pipeline setup complete!
To run: uncomment the main() call and execute with sufficient GPU memory


In [5]:
model, tokenizer, results = main()

Using device: cuda
Loading WikiText-2 dataset...
Train texts: 23767
Validation texts: 2461
Test texts: 2891
Setting up GPT-2 model and tokenizer...
Creating datasets...
Processing texts...


100%|██████████| 1000/1000 [00:01<00:00, 991.06it/s]


Processing texts...


100%|██████████| 200/200 [00:00<00:00, 1186.73it/s]


Processing texts...


...

Evaluating fine-tuned model...


Calculating perplexity: 100%|██████████| 27/27 [00:04<00:00,  6.08it/s]


Perplexity: 17153.29


Calculating top-k accuracy: 100%|██████████| 27/27 [05:05<00:00, 11.33s/it]


Top-k Accuracies:
  Top-1: 0.3728
  Top-5: 0.5901
  Top-10: 0.6654

Sample Predictions:
Input: 'Albert Einstein was a'
Predictions: [' analyst', ' for', ' the']
------------------------------
Input: 'The capital of France is'
Predictions: [' covered', ' by', ' ceilings']
------------------------------
Input: 'Machine learning is the'
Predictions: [' neural', ' process', ' that']
------------------------------
Input: 'In the year 2024, artificial intelligence'
Predictions: [' developed', ' a', ' vehicle']
------------------------------
Input: 'The quick brown fox'
Predictions: [' fired', ' off', ' a']
------------------------------

Final Results Summary:
         Metric    Value
     Perplexity 17153.29
 Top-1 Accuracy   0.3728
 Top-5 Accuracy   0.5901
Top-10 Accuracy   0.6654


In [9]:
# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from transformers import (
#     GPT2LMHeadModel, 
#     GPT2Tokenizer, 
#     TrainingArguments, 
#     Trainer,
#     DataCollatorForLanguageModeling
# )
# from datasets import load_dataset
# import numpy as np
# from tqdm import tqdm
# import matplotlib.pyplot as plt
# import pandas as pd
# from typing import Dict, List, Tuple
# import math

# class WikiTextDataset(Dataset):
#     """Custom dataset for WikiText-2 with proper tokenization"""
    
#     def __init__(self, texts: List[str], tokenizer, max_length: int = 512):
#         self.tokenizer = tokenizer
#         self.max_length = max_length
#         self.examples = []
        
#         print("Processing texts...")
#         for text in tqdm(texts):
#             if len(text.strip()) > 0:  # Skip empty texts
#                 # Tokenize and create sliding windows
#                 tokens = tokenizer.encode(text, add_special_tokens=True)
                
#                 # Create overlapping sequences
#                 for i in range(0, len(tokens) - 1, max_length // 2):
#                     chunk = tokens[i:i + max_length]
#                     if len(chunk) > 1:  # Need at least 2 tokens (input + target)
#                         self.examples.append(chunk)
    
#     def __len__(self):
#         return len(self.examples)
    
#     def __getitem__(self, idx):
#         return {
#             'input_ids': torch.tensor(self.examples[idx], dtype=torch.long),
#             'attention_mask': torch.ones(len(self.examples[idx]), dtype=torch.long)
#         }

# class NWPEvaluator:
#     """Evaluator for Next Word Prediction metrics"""
    
#     def __init__(self, model, tokenizer, device):
#         self.model = model
#         self.tokenizer = tokenizer
#         self.device = device
#         self.model.eval()
    
#     def calculate_perplexity(self, dataloader) -> float:
#         """Calculate perplexity on evaluation dataset"""
#         total_loss = 0
#         total_tokens = 0
        
#         with torch.no_grad():
#             for batch in tqdm(dataloader, desc="Calculating perplexity"):
#                 input_ids = batch['input_ids'].to(self.device)
#                 attention_mask = batch['attention_mask'].to(self.device)
                
#                 # Shift labels for next token prediction
#                 labels = input_ids.clone()
#                 labels[:, :-1] = input_ids[:, 1:]
#                 labels[:, -1] = -100  # Ignore last token
                
#                 outputs = self.model(
#                     input_ids=input_ids,
#                     attention_mask=attention_mask,
#                     labels=labels
#                 )
                
#                 loss = outputs.loss
#                 # Count valid tokens (not -100)
#                 valid_tokens = (labels != -100).sum().item()
                
#                 total_loss += loss.item() * valid_tokens
#                 total_tokens += valid_tokens
        
#         avg_loss = total_loss / total_tokens
#         perplexity = math.exp(avg_loss)
#         return perplexity
    
#     def calculate_top_k_accuracy(self, dataloader, k_values: List[int] = [1, 5, 10]) -> Dict[int, float]:
#         """Calculate top-k accuracy for next word prediction"""
#         correct_predictions = {k: 0 for k in k_values}
#         total_predictions = 0
        
#         with torch.no_grad():
#             for batch in tqdm(dataloader, desc="Calculating top-k accuracy"):
#                 input_ids = batch['input_ids'].to(self.device)
#                 attention_mask = batch['attention_mask'].to(self.device)
                
#                 for seq_idx in range(input_ids.size(0)):
#                     sequence = input_ids[seq_idx]
#                     mask = attention_mask[seq_idx]
                    
#                     # Get valid length
#                     valid_length = mask.sum().item()
                    
#                     # Predict each position (except last)
#                     for pos in range(1, valid_length):
#                         context = sequence[:pos].unsqueeze(0)
#                         target = sequence[pos].item()
                        
#                         outputs = self.model(context)
#                         logits = outputs.logits[0, -1, :]  # Last position logits
                        
#                         # Get top-k predictions
#                         top_k_max = max(k_values)
#                         top_k_tokens = torch.topk(logits, top_k_max).indices
                        
#                         # Check accuracy for different k values
#                         for k in k_values:
#                             if target in top_k_tokens[:k]:
#                                 correct_predictions[k] += 1
                        
#                         total_predictions += 1
        
#         # Calculate accuracies
#         accuracies = {k: correct_predictions[k] / total_predictions for k in k_values}
#         return accuracies
    
#     def sample_predictions(self, text: str, num_predictions: int = 5) -> List[str]:
#         """Generate sample predictions for demonstration"""
#         tokens = self.tokenizer.encode(text, return_tensors='pt').to(self.device)
        
#         predictions = []
#         with torch.no_grad():
#             for _ in range(num_predictions):
#                 outputs = self.model(tokens)
#                 logits = outputs.logits[0, -1, :]
                
#                 # Sample from distribution
#                 probs = torch.softmax(logits, dim=-1)
#                 next_token = torch.multinomial(probs, 1)
                
#                 predicted_word = self.tokenizer.decode(next_token.item())
#                 predictions.append(predicted_word)
                
#                 # Add to context for next prediction
#                 tokens = torch.cat([tokens, next_token.unsqueeze(0)], dim=1)
        
#         return predictions

# def load_and_prepare_data():
#     """Load WikiText-2 dataset and prepare for training"""
#     print("Loading WikiText-2 dataset...")
#     dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
    
#     # Filter out empty texts
#     train_texts = [text for text in dataset['train']['text'] if len(text.strip()) > 0]
#     valid_texts = [text for text in dataset['validation']['text'] if len(text.strip()) > 0]
#     test_texts = [text for text in dataset['test']['text'] if len(text.strip()) > 0]
    
#     print(f"Train texts: {len(train_texts)}")
#     print(f"Validation texts: {len(valid_texts)}")
#     print(f"Test texts: {len(test_texts)}")
    
#     return train_texts, valid_texts, test_texts

# def setup_model_and_tokenizer():
#     """Initialize GPT-2 model and tokenizer"""
#     print("Setting up GPT-2 model and tokenizer...")
#     model_name = "gpt2"
    
#     tokenizer = GPT2Tokenizer.from_pretrained(model_name)
#     model = GPT2LMHeadModel.from_pretrained(model_name)
    
#     # Add padding token
#     tokenizer.pad_token = tokenizer.eos_token
    
#     return model, tokenizer

# def fine_tune_model(model, tokenizer, train_dataset, eval_dataset, output_dir="./gpt2-wikitext2"):
#     """Fine-tune GPT-2 on WikiText-2"""
    
#     # Data collator for language modeling
#     data_collator = DataCollatorForLanguageModeling(
#         tokenizer=tokenizer,
#         mlm=False,  # GPT-2 is autoregressive, not masked LM
#         pad_to_multiple_of=8,
#         return_tensors="pt"
#     )
    
#     # Training arguments
#     training_args = TrainingArguments(
#         output_dir=output_dir,
#         overwrite_output_dir=True,
#         num_train_epochs=50,
#         per_device_train_batch_size=4,
#         per_device_eval_batch_size=4,
#         warmup_steps=500,
#         logging_steps=100,
#         save_steps=1000,
#         eval_steps=500,
#         eval_strategy="steps",  # Changed from evaluation_strategy
#         load_best_model_at_end=True,
#         save_total_limit=2,
#         gradient_accumulation_steps=2,
#         fp16=True,  # Mixed precision training
#         dataloader_pin_memory=True,
#         learning_rate=5e-5,
#         weight_decay=0.01,
#         adam_epsilon=1e-8,
#         max_grad_norm=1.0,
#         lr_scheduler_type="linear",
#         report_to=[],  # Changed from None to empty list
#     )
    
#     # Initialize trainer
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         data_collator=data_collator,
#         train_dataset=train_dataset,
#         eval_dataset=eval_dataset,
#         tokenizer=tokenizer,
#     )
    
#     # Train the model
#     print("Starting fine-tuning...")
#     trainer.train()
    
#     # Save the fine-tuned model
#     trainer.save_model()
#     tokenizer.save_pretrained(output_dir)
    
#     return trainer

# def evaluate_model(model, tokenizer, test_dataset, device):
#     """Comprehensive evaluation of the fine-tuned model"""
#     print("Evaluating fine-tuned model...")
    
#     # Create test dataloader
#     test_dataloader = DataLoader(
#         test_dataset, 
#         batch_size=8, 
#         shuffle=False,
#         collate_fn=lambda x: {
#             'input_ids': torch.nn.utils.rnn.pad_sequence(
#                 [item['input_ids'] for item in x], 
#                 batch_first=True, 
#                 padding_value=tokenizer.pad_token_id
#             ),
#             'attention_mask': torch.nn.utils.rnn.pad_sequence(
#                 [item['attention_mask'] for item in x], 
#                 batch_first=True, 
#                 padding_value=0
#             )
#         }
#     )
    
#     # Initialize evaluator
#     evaluator = NWPEvaluator(model, tokenizer, device)
    
#     # Calculate perplexity
#     perplexity = evaluator.calculate_perplexity(test_dataloader)
#     print(f"Perplexity: {perplexity:.2f}")
    
#     # Calculate top-k accuracy
#     accuracies = evaluator.calculate_top_k_accuracy(test_dataloader, k_values=[1, 5, 10])
#     print("Top-k Accuracies:")
#     for k, acc in accuracies.items():
#         print(f"  Top-{k}: {acc:.4f}")
    
#     return perplexity, accuracies, evaluator

# def inspect_training_examples(dataset, tokenizer, num_examples=5, max_display_length=100):
#     """
#     Inspect and visualize training examples to understand the data format
    
#     Args:
#         dataset: WikiTextDataset instance
#         tokenizer: GPT2Tokenizer instance
#         num_examples: Number of examples to display
#         max_display_length: Maximum length of text to display
#     """
#     print("=" * 80)
#     print("TRAINING EXAMPLES INSPECTION")
#     print("=" * 80)
    
#     print(f"Dataset Info:")
#     print(f"  Total examples: {len(dataset)}")
#     print(f"  Max sequence length: {dataset.max_length}")
#     print(f"  Tokenizer vocab size: {len(tokenizer)}")
#     print(f"  Tokenizer pad token: '{tokenizer.pad_token}' (ID: {tokenizer.pad_token_id})")
#     print(f"  Tokenizer eos token: '{tokenizer.eos_token}' (ID: {tokenizer.eos_token_id})")
#     print()
    
#     for i in range(min(num_examples, len(dataset))):
#         print(f"EXAMPLE {i+1}:")
#         print("-" * 60)
        
#         # Get the raw example
#         example = dataset[i]
#         input_ids = example['input_ids']
#         attention_mask = example['attention_mask']
        
#         # Basic statistics
#         print(f"Sequence length: {len(input_ids)}")
#         print(f"Attention mask sum: {attention_mask.sum().item()}")
#         print()
        
#         # Convert tokens back to text
#         full_text = tokenizer.decode(input_ids, skip_special_tokens=False)
#         clean_text = tokenizer.decode(input_ids, skip_special_tokens=True)
        
#         # Display text (truncated if too long)
#         if len(full_text) > max_display_length:
#             display_text = full_text[:max_display_length] + "..."
#             clean_display = clean_text[:max_display_length] + "..."
#         else:
#             display_text = full_text
#             clean_display = clean_text
        
#         print(f"Full text (with special tokens):")
#         print(f"  '{display_text}'")
#         print()
#         print(f"Clean text (without special tokens):")
#         print(f"  '{clean_display}'")
#         print()
        
#         # Show first 20 tokens in detail
#         print("First 20 tokens breakdown:")
#         print("  Token ID | Token Text | Attention")
#         print("  ---------|------------|----------")
        
#         for j in range(min(20, len(input_ids))):
#             token_id = input_ids[j].item()
#             token_text = tokenizer.decode([token_id])
#             attention = attention_mask[j].item()
            
#             # Handle special characters for display
#             if token_text == tokenizer.eos_token:
#                 token_display = "<EOS>"
#             elif token_text == tokenizer.pad_token:
#                 token_display = "<PAD>"
#             elif token_text.strip() == "":
#                 token_display = "<SPACE>"
#             else:
#                 token_display = repr(token_text)
            
#             print(f"  {token_id:8d} | {token_display:10s} | {attention:9d}")
        
#         if len(input_ids) > 20:
#             print(f"  ... ({len(input_ids) - 20} more tokens)")
        
#         print()
#         print("=" * 60)
#         print()

# def show_next_word_prediction_examples(dataset, tokenizer, num_examples=3):
#     """
#     Show how the training examples work for next word prediction
    
#     Args:
#         dataset: WikiTextDataset instance
#         tokenizer: GPT2Tokenizer instance
#         num_examples: Number of examples to show
#     """
#     print("=" * 80)
#     print("NEXT WORD PREDICTION TRAINING FORMAT")
#     print("=" * 80)
    
#     print("This shows how each sequence is used for next word prediction training.")
#     print("For each position, the model sees all previous tokens and predicts the next one.")
#     print()
    
#     for i in range(min(num_examples, len(dataset))):
#         print(f"EXAMPLE {i+1}:")
#         print("-" * 60)
        
#         example = dataset[i]
#         input_ids = example['input_ids']
        
#         # Show first 10 positions as input -> target pairs
#         print("Input Context -> Target Token")
#         print("-" * 40)
        
#         for pos in range(1, min(11, len(input_ids))):
#             # Context is everything before current position
#             context_ids = input_ids[:pos]
#             target_id = input_ids[pos]
            
#             # Convert to text
#             context_text = tokenizer.decode(context_ids, skip_special_tokens=True)
#             target_text = tokenizer.decode([target_id], skip_special_tokens=True)
            
#             # Truncate context if too long
#             if len(context_text) > 50:
#                 context_display = "..." + context_text[-47:]
#             else:
#                 context_display = context_text
            
#             print(f"'{context_display}' -> '{target_text}'")
        
#         if len(input_ids) > 11:
#             print(f"... ({len(input_ids) - 11} more prediction pairs)")
        
#         print()
#         print("=" * 60)
#         print()

# def analyze_dataset_statistics(dataset, tokenizer):
#     """
#     Analyze statistical properties of the dataset
    
#     Args:
#         dataset: WikiTextDataset instance
#         tokenizer: GPT2Tokenizer instance
#     """
#     print("=" * 80)
#     print("DATASET STATISTICAL ANALYSIS")
#     print("=" * 80)
    
#     sequence_lengths = []
#     token_counts = {}
    
#     print("Analyzing dataset... (this may take a moment)")
    
#     for i in tqdm(range(len(dataset)), desc="Processing examples"):
#         example = dataset[i]
#         input_ids = example['input_ids']
        
#         # Collect sequence length
#         sequence_lengths.append(len(input_ids))
        
#         # Count token frequencies (sample first 1000 examples to save time)
#         if i < 1000:
#             for token_id in input_ids:
#                 token_id = token_id.item()
#                 token_counts[token_id] = token_counts.get(token_id, 0) + 1
    
#     # Sequence length statistics
#     sequence_lengths = np.array(sequence_lengths)
#     print(f"Sequence Length Statistics:")
#     print(f"  Mean: {sequence_lengths.mean():.2f}")
#     print(f"  Median: {np.median(sequence_lengths):.2f}")
#     print(f"  Min: {sequence_lengths.min()}")
#     print(f"  Max: {sequence_lengths.max()}")
#     print(f"  Std: {sequence_lengths.std():.2f}")
#     print()
    
#     # Token frequency analysis (top 20 most common tokens)
#     if token_counts:
#         print("Top 20 Most Common Tokens (from first 1000 examples):")
#         print("  Rank | Token ID | Count | Token Text")
#         print("  -----|----------|-------|----------")
        
#         sorted_tokens = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
#         for rank, (token_id, count) in enumerate(sorted_tokens[:20], 1):
#             token_text = tokenizer.decode([token_id])
            
#             # Handle special characters
#             if token_text == tokenizer.eos_token:
#                 token_display = "<EOS>"
#             elif token_text == tokenizer.pad_token:
#                 token_display = "<PAD>"
#             elif token_text.strip() == "":
#                 token_display = "<SPACE>"
#             else:
#                 token_display = repr(token_text)
            
#             print(f"  {rank:4d} | {token_id:8d} | {count:5d} | {token_display}")
        
#         print(f"\nTotal unique tokens in sample: {len(token_counts)}")
    
#     print()
#     print("=" * 80)

# def demonstrate_predictions(evaluator, tokenizer):
#     """Demonstrate model predictions on sample texts"""
#     print("\nSample Predictions:")
#     print("=" * 50)
    
#     sample_texts = [
#         "Albert Einstein was a",
#         "The capital of France is",
#         "Machine learning is the",
#         "In the year 2024, artificial intelligence",
#         "The quick brown fox"
#     ]
    
#     for text in sample_texts:
#         predictions = evaluator.sample_predictions(text, num_predictions=3)
#         print(f"Input: '{text}'")
#         print(f"Predictions: {predictions}")
#         print("-" * 30)

# def main():
#     # Set device
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"Using device: {device}")
    
#     # Load data
#     train_texts, valid_texts, test_texts = load_and_prepare_data()
    
#     # Setup model and tokenizer
#     model, tokenizer = setup_model_and_tokenizer()
#     model.to(device)
    
#     # Create datasets
#     print("Creating datasets...")
#     train_dataset = WikiTextDataset(train_texts[:1000], tokenizer)  # Subset for demo
#     eval_dataset = WikiTextDataset(valid_texts[:200], tokenizer)
#     test_dataset = WikiTextDataset(test_texts[:200], tokenizer)
    
#     # Fine-tune model
#     trainer = fine_tune_model(model, tokenizer, train_dataset, eval_dataset)
    
#     # Evaluate model
#     perplexity, accuracies, evaluator = evaluate_model(model, tokenizer, test_dataset, device)
    
#     # Demonstrate predictions
#     demonstrate_predictions(evaluator, tokenizer)
    
#     # Create results summary
#     results = {
#         'Metric': ['Perplexity', 'Top-1 Accuracy', 'Top-5 Accuracy', 'Top-10 Accuracy'],
#         'Value': [f"{perplexity:.2f}", f"{accuracies[1]:.4f}", f"{accuracies[5]:.4f}", f"{accuracies[10]:.4f}"]
#     }
    
#     results_df = pd.DataFrame(results)
#     print("\nFinal Results Summary:")
#     print(results_df.to_string(index=False))
    
#     return model, tokenizer, results_df

# def explore_training_data():
#     """
#     Comprehensive exploration of training data
#     Run this before training to understand your dataset
#     """
#     print("🔍 EXPLORING TRAINING DATA")
#     print("=" * 80)
    
#     # Load a small sample of data for exploration
#     print("Loading sample data...")
#     train_texts, valid_texts, test_texts = load_and_prepare_data()
    
#     # Setup tokenizer
#     model, tokenizer = setup_model_and_tokenizer()
    
#     # Create small datasets for exploration
#     sample_train = WikiTextDataset(train_texts[:50], tokenizer)  # Small sample
#     sample_valid = WikiTextDataset(valid_texts[:20], tokenizer)
    
#     print(f"Created sample datasets:")
#     print(f"  Sample train: {len(sample_train)} examples")
#     print(f"  Sample valid: {len(sample_valid)} examples")
#     print()
    
#     # 1. Inspect raw training examples
#     print("1️⃣ INSPECTING RAW TRAINING EXAMPLES")
#     inspect_training_examples(sample_train, tokenizer, num_examples=3)
    
#     # 2. Show next word prediction format
#     print("2️⃣ NEXT WORD PREDICTION FORMAT")
#     show_next_word_prediction_examples(sample_train, tokenizer, num_examples=2)
    
#     # 3. Dataset statistics
#     print("3️⃣ DATASET STATISTICS")
#     analyze_dataset_statistics(sample_train, tokenizer)
    
#     # 4. Show some original WikiText content
#     print("4️⃣ SAMPLE WIKITEXT CONTENT")
#     print("=" * 40)
#     print("Here are some original texts from WikiText-2:")
#     print()
    
#     for i, text in enumerate(train_texts[:3]):
#         if len(text.strip()) > 0:
#             print(f"Text {i+1}:")
#             print(f"  Length: {len(text)} characters")
#             print(f"  Preview: {text[:200]}...")
#             print()
    
#     return sample_train, sample_valid, tokenizer

# if __name__ == "__main__":
#     # Note: This is a comprehensive implementation
#     # For actual execution, you may want to run sections separately
#     # due to computational requirements
    
#     print("GPT-2 Fine-tuning Pipeline for Next Word Prediction")
#     print("=" * 60)
    
#     # Uncomment to explore training data first
#     # sample_train, sample_valid, tokenizer = explore_training_data()
    
#     # Uncomment the following line to run the full pipeline
#     model, tokenizer, results = main()
    
#     print("Pipeline setup complete!")
#     print("To explore data: uncomment explore_training_data() call")
#     print("To run training: uncomment the main() call and execute with sufficient GPU memory")

GPT-2 Fine-tuning Pipeline for Next Word Prediction
Using device: cuda
Loading WikiText-2 dataset...
Train texts: 23767
Validation texts: 2461
Test texts: 2891
Setting up GPT-2 model and tokenizer...
Creating datasets...
Processing texts...


100%|██████████| 1000/1000 [00:00<00:00, 1044.33it/s]


Processing texts...


100%|██████████| 200/200 [00:00<00:00, 1230.62it/s]


Processing texts...


...

Evaluating fine-tuned model...


Calculating perplexity: 100%|██████████| 27/27 [00:04<00:00,  6.02it/s]


Perplexity: 262906461.75


Calculating top-k accuracy: 100%|██████████| 27/27 [05:05<00:00, 11.30s/it]

Top-k Accuracies:
  Top-1: 0.3412
  Top-5: 0.5427
  Top-10: 0.6134

Sample Predictions:
Input: 'Albert Einstein was a'
Predictions: [' devout', ' Catholic', ' and']
------------------------------
Input: 'The capital of France is'
Predictions: [' Br', 'ind', 'isi']
------------------------------
Input: 'Machine learning is the'
Predictions: [' art', ' of', ' training']
------------------------------
Input: 'In the year 2024, artificial intelligence'
Predictions: [' (', ' AI', ' )']
------------------------------
Input: 'The quick brown fox'
Predictions: [' was', ' once', ' believed']
------------------------------

Final Results Summary:
         Metric        Value
     Perplexity 262906461.75
 Top-1 Accuracy       0.3412
 Top-5 Accuracy       0.5427
Top-10 Accuracy       0.6134
Pipeline setup complete!
To explore data: uncomment explore_training_data() call
To run training: uncomment the main() call and execute with sufficient GPU memory


In [8]:
# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from transformers import (
#     GPT2LMHeadModel, 
#     GPT2Tokenizer, 
#     TrainingArguments, 
#     Trainer,
#     DataCollatorForLanguageModeling
# )
# from datasets import load_dataset
# import numpy as np
# from tqdm import tqdm
# import matplotlib.pyplot as plt
# import pandas as pd
# from typing import Dict, List, Tuple
# import math

# class WikiTextDataset(Dataset):
#     """Custom dataset for WikiText-2 with proper tokenization"""
    
#     def __init__(self, texts: List[str], tokenizer, max_length: int = 512):
#         self.tokenizer = tokenizer
#         self.max_length = max_length
#         self.examples = []
        
#         print("Processing texts...")
#         for text in tqdm(texts):
#             if len(text.strip()) > 0:  # Skip empty texts
#                 # Tokenize and create sliding windows
#                 tokens = tokenizer.encode(text, add_special_tokens=True)
                
#                 # Create overlapping sequences
#                 for i in range(0, len(tokens) - 1, max_length // 2):
#                     chunk = tokens[i:i + max_length]
#                     if len(chunk) > 1:  # Need at least 2 tokens (input + target)
#                         self.examples.append(chunk)
    
#     def __len__(self):
#         return len(self.examples)
    
#     def __getitem__(self, idx):
#         return {
#             'input_ids': torch.tensor(self.examples[idx], dtype=torch.long),
#             'attention_mask': torch.ones(len(self.examples[idx]), dtype=torch.long)
#         }

# class NWPEvaluator:
#     """Evaluator for Next Word Prediction metrics"""
    
#     def __init__(self, model, tokenizer, device):
#         self.model = model
#         self.tokenizer = tokenizer
#         self.device = device
#         self.model.eval()
    
#     def calculate_perplexity(self, dataloader) -> float:
#         """Calculate perplexity on evaluation dataset"""
#         total_loss = 0
#         total_tokens = 0
        
#         with torch.no_grad():
#             for batch in tqdm(dataloader, desc="Calculating perplexity"):
#                 input_ids = batch['input_ids'].to(self.device)
#                 attention_mask = batch['attention_mask'].to(self.device)
                
#                 # Shift labels for next token prediction
#                 labels = input_ids.clone()
#                 labels[:, :-1] = input_ids[:, 1:]
#                 labels[:, -1] = -100  # Ignore last token
                
#                 outputs = self.model(
#                     input_ids=input_ids,
#                     attention_mask=attention_mask,
#                     labels=labels
#                 )
                
#                 loss = outputs.loss
#                 # Count valid tokens (not -100)
#                 valid_tokens = (labels != -100).sum().item()
                
#                 total_loss += loss.item() * valid_tokens
#                 total_tokens += valid_tokens
        
#         avg_loss = total_loss / total_tokens
#         perplexity = math.exp(avg_loss)
#         return perplexity
    
#     def calculate_top_k_accuracy(self, dataloader, k_values: List[int] = [1, 5, 10]) -> Dict[int, float]:
#         """Calculate top-k accuracy for next word prediction"""
#         correct_predictions = {k: 0 for k in k_values}
#         total_predictions = 0
        
#         with torch.no_grad():
#             for batch in tqdm(dataloader, desc="Calculating top-k accuracy"):
#                 input_ids = batch['input_ids'].to(self.device)
#                 attention_mask = batch['attention_mask'].to(self.device)
                
#                 for seq_idx in range(input_ids.size(0)):
#                     sequence = input_ids[seq_idx]
#                     mask = attention_mask[seq_idx]
                    
#                     # Get valid length
#                     valid_length = mask.sum().item()
                    
#                     # Predict each position (except last)
#                     for pos in range(1, valid_length):
#                         context = sequence[:pos].unsqueeze(0)
#                         target = sequence[pos].item()
                        
#                         outputs = self.model(context)
#                         logits = outputs.logits[0, -1, :]  # Last position logits
                        
#                         # Get top-k predictions
#                         top_k_max = max(k_values)
#                         top_k_tokens = torch.topk(logits, top_k_max).indices
                        
#                         # Check accuracy for different k values
#                         for k in k_values:
#                             if target in top_k_tokens[:k]:
#                                 correct_predictions[k] += 1
                        
#                         total_predictions += 1
        
#         # Calculate accuracies
#         accuracies = {k: correct_predictions[k] / total_predictions for k in k_values}
#         return accuracies
    
#     def sample_predictions(self, text: str, num_predictions: int = 5) -> List[str]:
#         """Generate sample predictions for demonstration"""
#         tokens = self.tokenizer.encode(text, return_tensors='pt').to(self.device)
        
#         predictions = []
#         with torch.no_grad():
#             for _ in range(num_predictions):
#                 outputs = self.model(tokens)
#                 logits = outputs.logits[0, -1, :]
                
#                 # Sample from distribution
#                 probs = torch.softmax(logits, dim=-1)
#                 next_token = torch.multinomial(probs, 1)
                
#                 predicted_word = self.tokenizer.decode(next_token.item())
#                 predictions.append(predicted_word)
                
#                 # Add to context for next prediction
#                 tokens = torch.cat([tokens, next_token.unsqueeze(0)], dim=1)
        
#         return predictions

# def load_and_prepare_data():
#     """Load WikiText-2 dataset and prepare for training"""
#     print("Loading WikiText-2 dataset...")
#     dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
    
#     # Filter out empty texts
#     train_texts = [text for text in dataset['train']['text'] if len(text.strip()) > 0]
#     valid_texts = [text for text in dataset['validation']['text'] if len(text.strip()) > 0]
#     test_texts = [text for text in dataset['test']['text'] if len(text.strip()) > 0]
    
#     print(f"Train texts: {len(train_texts)}")
#     print(f"Validation texts: {len(valid_texts)}")
#     print(f"Test texts: {len(test_texts)}")
    
#     return train_texts, valid_texts, test_texts

# def setup_model_and_tokenizer():
#     """Initialize GPT-2 model and tokenizer"""
#     print("Setting up GPT-2 model and tokenizer...")
#     model_name = "gpt2"
    
#     tokenizer = GPT2Tokenizer.from_pretrained(model_name)
#     model = GPT2LMHeadModel.from_pretrained(model_name)
    
#     # Add padding token
#     tokenizer.pad_token = tokenizer.eos_token
    
#     return model, tokenizer

# def fine_tune_model(model, tokenizer, train_dataset, eval_dataset, output_dir="./gpt2-wikitext2"):
#     """Fine-tune GPT-2 on WikiText-2"""
    
#     # Data collator for language modeling
#     data_collator = DataCollatorForLanguageModeling(
#         tokenizer=tokenizer,
#         mlm=False,  # GPT-2 is autoregressive, not masked LM
#         pad_to_multiple_of=8,
#         return_tensors="pt"
#     )
    
#     # Training arguments
#     training_args = TrainingArguments(
#         output_dir=output_dir,
#         overwrite_output_dir=True,
#         num_train_epochs=3,
#         per_device_train_batch_size=4,
#         per_device_eval_batch_size=4,
#         warmup_steps=500,
#         logging_steps=100,
#         save_steps=1000,
#         eval_steps=500,
#         eval_strategy="steps",  # Changed from evaluation_strategy
#         load_best_model_at_end=True,
#         save_total_limit=2,
#         gradient_accumulation_steps=2,
#         fp16=True,  # Mixed precision training
#         dataloader_pin_memory=True,
#         learning_rate=5e-5,
#         weight_decay=0.01,
#         adam_epsilon=1e-8,
#         max_grad_norm=1.0,
#         lr_scheduler_type="linear",
#         report_to=[],  # Changed from None to empty list
#     )
    
#     # Initialize trainer
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         data_collator=data_collator,
#         train_dataset=train_dataset,
#         eval_dataset=eval_dataset,
#         tokenizer=tokenizer,
#     )
    
#     # Train the model
#     print("Starting fine-tuning...")
#     trainer.train()
    
#     # Save the fine-tuned model
#     trainer.save_model()
#     tokenizer.save_pretrained(output_dir)
    
#     return trainer

# def evaluate_model(model, tokenizer, test_dataset, device):
#     """Comprehensive evaluation of the fine-tuned model"""
#     print("Evaluating fine-tuned model...")
    
#     # Create test dataloader
#     test_dataloader = DataLoader(
#         test_dataset, 
#         batch_size=8, 
#         shuffle=False,
#         collate_fn=lambda x: {
#             'input_ids': torch.nn.utils.rnn.pad_sequence(
#                 [item['input_ids'] for item in x], 
#                 batch_first=True, 
#                 padding_value=tokenizer.pad_token_id
#             ),
#             'attention_mask': torch.nn.utils.rnn.pad_sequence(
#                 [item['attention_mask'] for item in x], 
#                 batch_first=True, 
#                 padding_value=0
#             )
#         }
#     )
    
#     # Initialize evaluator
#     evaluator = NWPEvaluator(model, tokenizer, device)
    
#     # Calculate perplexity
#     perplexity = evaluator.calculate_perplexity(test_dataloader)
#     print(f"Perplexity: {perplexity:.2f}")
    
#     # Calculate top-k accuracy
#     accuracies = evaluator.calculate_top_k_accuracy(test_dataloader, k_values=[1, 5, 10])
#     print("Top-k Accuracies:")
#     for k, acc in accuracies.items():
#         print(f"  Top-{k}: {acc:.4f}")
    
#     return perplexity, accuracies, evaluator

# def inspect_training_examples(dataset, tokenizer, num_examples=5, max_display_length=100):
#     """
#     Inspect and visualize training examples to understand the data format
    
#     Args:
#         dataset: WikiTextDataset instance
#         tokenizer: GPT2Tokenizer instance
#         num_examples: Number of examples to display
#         max_display_length: Maximum length of text to display
#     """
#     print("=" * 80)
#     print("TRAINING EXAMPLES INSPECTION")
#     print("=" * 80)
    
#     print(f"Dataset Info:")
#     print(f"  Total examples: {len(dataset)}")
#     print(f"  Max sequence length: {dataset.max_length}")
#     print(f"  Tokenizer vocab size: {len(tokenizer)}")
#     print(f"  Tokenizer pad token: '{tokenizer.pad_token}' (ID: {tokenizer.pad_token_id})")
#     print(f"  Tokenizer eos token: '{tokenizer.eos_token}' (ID: {tokenizer.eos_token_id})")
#     print()
    
#     for i in range(min(num_examples, len(dataset))):
#         print(f"EXAMPLE {i+1}:")
#         print("-" * 60)
        
#         # Get the raw example
#         example = dataset[i]
#         input_ids = example['input_ids']
#         attention_mask = example['attention_mask']
        
#         # Basic statistics
#         print(f"Sequence length: {len(input_ids)}")
#         print(f"Attention mask sum: {attention_mask.sum().item()}")
#         print()
        
#         # Convert tokens back to text
#         full_text = tokenizer.decode(input_ids, skip_special_tokens=False)
#         clean_text = tokenizer.decode(input_ids, skip_special_tokens=True)
        
#         # Display text (truncated if too long)
#         if len(full_text) > max_display_length:
#             display_text = full_text[:max_display_length] + "..."
#             clean_display = clean_text[:max_display_length] + "..."
#         else:
#             display_text = full_text
#             clean_display = clean_text
        
#         print(f"Full text (with special tokens):")
#         print(f"  '{display_text}'")
#         print()
#         print(f"Clean text (without special tokens):")
#         print(f"  '{clean_display}'")
#         print()
        
#         # Show first 20 tokens in detail
#         print("First 20 tokens breakdown:")
#         print("  Token ID | Token Text | Attention")
#         print("  ---------|------------|----------")
        
#         for j in range(min(20, len(input_ids))):
#             token_id = input_ids[j].item()
#             token_text = tokenizer.decode([token_id])
#             attention = attention_mask[j].item()
            
#             # Handle special characters for display
#             if token_text == tokenizer.eos_token:
#                 token_display = "<EOS>"
#             elif token_text == tokenizer.pad_token:
#                 token_display = "<PAD>"
#             elif token_text.strip() == "":
#                 token_display = "<SPACE>"
#             else:
#                 token_display = repr(token_text)
            
#             print(f"  {token_id:8d} | {token_display:10s} | {attention:9d}")
        
#         if len(input_ids) > 20:
#             print(f"  ... ({len(input_ids) - 20} more tokens)")
        
#         print()
#         print("=" * 60)
#         print()

# def show_next_word_prediction_examples(dataset, tokenizer, num_examples=3):
#     """
#     Show how the training examples work for next word prediction
    
#     Args:
#         dataset: WikiTextDataset instance
#         tokenizer: GPT2Tokenizer instance
#         num_examples: Number of examples to show
#     """
#     print("=" * 80)
#     print("NEXT WORD PREDICTION TRAINING FORMAT")
#     print("=" * 80)
    
#     print("This shows how each sequence is used for next word prediction training.")
#     print("For each position, the model sees all previous tokens and predicts the next one.")
#     print()
    
#     for i in range(min(num_examples, len(dataset))):
#         print(f"EXAMPLE {i+1}:")
#         print("-" * 60)
        
#         example = dataset[i]
#         input_ids = example['input_ids']
        
#         # Show first 10 positions as input -> target pairs
#         print("Input Context -> Target Token")
#         print("-" * 40)
        
#         for pos in range(1, min(11, len(input_ids))):
#             # Context is everything before current position
#             context_ids = input_ids[:pos]
#             target_id = input_ids[pos]
            
#             # Convert to text
#             context_text = tokenizer.decode(context_ids, skip_special_tokens=True)
#             target_text = tokenizer.decode([target_id], skip_special_tokens=True)
            
#             # Truncate context if too long
#             if len(context_text) > 50:
#                 context_display = "..." + context_text[-47:]
#             else:
#                 context_display = context_text
            
#             print(f"'{context_display}' -> '{target_text}'")
        
#         if len(input_ids) > 11:
#             print(f"... ({len(input_ids) - 11} more prediction pairs)")
        
#         print()
#         print("=" * 60)
#         print()

# def analyze_dataset_statistics(dataset, tokenizer):
#     """
#     Analyze statistical properties of the dataset
    
#     Args:
#         dataset: WikiTextDataset instance
#         tokenizer: GPT2Tokenizer instance
#     """
#     print("=" * 80)
#     print("DATASET STATISTICAL ANALYSIS")
#     print("=" * 80)
    
#     sequence_lengths = []
#     token_counts = {}
    
#     print("Analyzing dataset... (this may take a moment)")
    
#     for i in tqdm(range(len(dataset)), desc="Processing examples"):
#         example = dataset[i]
#         input_ids = example['input_ids']
        
#         # Collect sequence length
#         sequence_lengths.append(len(input_ids))
        
#         # Count token frequencies (sample first 1000 examples to save time)
#         if i < 1000:
#             for token_id in input_ids:
#                 token_id = token_id.item()
#                 token_counts[token_id] = token_counts.get(token_id, 0) + 1
    
#     # Sequence length statistics
#     sequence_lengths = np.array(sequence_lengths)
#     print(f"Sequence Length Statistics:")
#     print(f"  Mean: {sequence_lengths.mean():.2f}")
#     print(f"  Median: {np.median(sequence_lengths):.2f}")
#     print(f"  Min: {sequence_lengths.min()}")
#     print(f"  Max: {sequence_lengths.max()}")
#     print(f"  Std: {sequence_lengths.std():.2f}")
#     print()
    
#     # Token frequency analysis (top 20 most common tokens)
#     if token_counts:
#         print("Top 20 Most Common Tokens (from first 1000 examples):")
#         print("  Rank | Token ID | Count | Token Text")
#         print("  -----|----------|-------|----------")
        
#         sorted_tokens = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
#         for rank, (token_id, count) in enumerate(sorted_tokens[:20], 1):
#             token_text = tokenizer.decode([token_id])
            
#             # Handle special characters
#             if token_text == tokenizer.eos_token:
#                 token_display = "<EOS>"
#             elif token_text == tokenizer.pad_token:
#                 token_display = "<PAD>"
#             elif token_text.strip() == "":
#                 token_display = "<SPACE>"
#             else:
#                 token_display = repr(token_text)
            
#             print(f"  {rank:4d} | {token_id:8d} | {count:5d} | {token_display}")
        
#         print(f"\nTotal unique tokens in sample: {len(token_counts)}")
    
#     print()
#     print("=" * 80)

# def demonstrate_predictions(evaluator, tokenizer):
#     """Demonstrate model predictions on sample texts"""
#     print("\nSample Predictions:")
#     print("=" * 50)
    
#     sample_texts = [
#         "Albert Einstein was a",
#         "The capital of France is",
#         "Machine learning is the",
#         "In the year 2024, artificial intelligence",
#         "The quick brown fox"
#     ]
    
#     for text in sample_texts:
#         predictions = evaluator.sample_predictions(text, num_predictions=3)
#         print(f"Input: '{text}'")
#         print(f"Predictions: {predictions}")
#         print("-" * 30)

# def main():
#     # Set device
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"Using device: {device}")
    
#     # Load data
#     train_texts, valid_texts, test_texts = load_and_prepare_data()
    
#     # Setup model and tokenizer
#     model, tokenizer = setup_model_and_tokenizer()
#     model.to(device)
    
#     # Create datasets
#     print("Creating datasets...")
#     train_dataset = WikiTextDataset(train_texts[:1000], tokenizer)  # Subset for demo
#     eval_dataset = WikiTextDataset(valid_texts[:200], tokenizer)
#     test_dataset = WikiTextDataset(test_texts[:200], tokenizer)
    
#     # Fine-tune model
#     trainer = fine_tune_model(model, tokenizer, train_dataset, eval_dataset)
    
#     # Evaluate model
#     perplexity, accuracies, evaluator = evaluate_model(model, tokenizer, test_dataset, device)
    
#     # Demonstrate predictions
#     demonstrate_predictions(evaluator, tokenizer)
    
#     # Create results summary
#     results = {
#         'Metric': ['Perplexity', 'Top-1 Accuracy', 'Top-5 Accuracy', 'Top-10 Accuracy'],
#         'Value': [f"{perplexity:.2f}", f"{accuracies[1]:.4f}", f"{accuracies[5]:.4f}", f"{accuracies[10]:.4f}"]
#     }
    
#     results_df = pd.DataFrame(results)
#     print("\nFinal Results Summary:")
#     print(results_df.to_string(index=False))
    
#     return model, tokenizer, results_df

# def explore_training_data():
#     """
#     Comprehensive exploration of training data
#     Run this before training to understand your dataset
#     """
#     print("🔍 EXPLORING TRAINING DATA")
#     print("=" * 80)
    
#     # Load a small sample of data for exploration
#     print("Loading sample data...")
#     train_texts, valid_texts, test_texts = load_and_prepare_data()
    
#     # Setup tokenizer
#     model, tokenizer = setup_model_and_tokenizer()
    
#     # Create small datasets for exploration
#     sample_train = WikiTextDataset(train_texts[:50], tokenizer)  # Small sample
#     sample_valid = WikiTextDataset(valid_texts[:20], tokenizer)
    
#     print(f"Created sample datasets:")
#     print(f"  Sample train: {len(sample_train)} examples")
#     print(f"  Sample valid: {len(sample_valid)} examples")
#     print()
    
#     # 1. Inspect raw training examples
#     print("1️⃣ INSPECTING RAW TRAINING EXAMPLES")
#     inspect_training_examples(sample_train, tokenizer, num_examples=3)
    
#     # 2. Show next word prediction format
#     print("2️⃣ NEXT WORD PREDICTION FORMAT")
#     show_next_word_prediction_examples(sample_train, tokenizer, num_examples=2)
    
#     # 3. Dataset statistics
#     print("3️⃣ DATASET STATISTICS")
#     analyze_dataset_statistics(sample_train, tokenizer)
    
#     # 4. Show some original WikiText content
#     print("4️⃣ SAMPLE WIKITEXT CONTENT")
#     print("=" * 40)
#     print("Here are some original texts from WikiText-2:")
#     print()
    
#     for i, text in enumerate(train_texts[:3]):
#         if len(text.strip()) > 0:
#             print(f"Text {i+1}:")
#             print(f"  Length: {len(text)} characters")
#             print(f"  Preview: {text[:200]}...")
#             print()
    
#     return sample_train, sample_valid, tokenizer

# if __name__ == "__main__":
#     # Note: This is a comprehensive implementation
#     # For actual execution, you may want to run sections separately
#     # due to computational requirements
    
#     print("GPT-2 Fine-tuning Pipeline for Next Word Prediction")
#     print("=" * 60)
    
#     # Uncomment to explore training data first
#     sample_train, sample_valid, tokenizer = explore_training_data()
    
#     # Uncomment the following line to run the full pipeline
#     # model, tokenizer, results = main()
    
#     print("Pipeline setup complete!")
#     print("To explore data: uncomment explore_training_data() call")
#     print("To run training: uncomment the main() call and execute with sufficient GPU memory")

GPT-2 Fine-tuning Pipeline for Next Word Prediction
🔍 EXPLORING TRAINING DATA
Loading sample data...
Loading WikiText-2 dataset...
Train texts: 23767
Validation texts: 2461
Test texts: 2891
Setting up GPT-2 model and tokenizer...
Processing texts...


100%|██████████| 50/50 [00:00<00:00, 655.82it/s]


Processing texts...


100%|██████████| 20/20 [00:00<00:00, 1216.30it/s]


Created sample datasets:
  Sample train: 58 examples
  Sample valid: 20 examples

1️⃣ INSPECTING RAW TRAINING EXAMPLES
TRAINING EXAMPLES INSPECTION
Dataset Info:
  Total examples: 58
  Max sequence length: 512
  Tokenizer vocab size: 50257
  Tokenizer pad token: '<|endoftext|>' (ID: 50256)
  Tokenizer eos token: '<|endoftext|>' (ID: 50256)

EXAMPLE 1:
------------------------------------------------------------
Sequence length: 9
Attention mask sum: 9

Full text (with special tokens):
  ' = Valkyria Chronicles III = 
'

Clean text (without special tokens):
  ' = Valkyria Chronicles III = 
'

First 20 tokens breakdown:
  Token ID | Token Text | Attention
  ---------|------------|----------
       796 | ' ='       |         1
       569 | ' V'       |         1
     18354 | 'alky'     |         1
      7496 | 'ria'      |         1
     17740 | ' Chronicles' |         1
      6711 | ' III'     |         1
       796 | ' ='       |         1
       220 | <SPACE>    |         1
       198 

Processing examples: 100%|██████████| 58/58 [00:00<00:00, 4561.25it/s]

Sequence Length Statistics:
  Mean: 110.33
  Median: 85.50
  Min: 7
  Max: 351
  Std: 102.88

Top 20 Most Common Tokens (from first 1000 examples):
  Rank | Token ID | Count | Token Text
  -----|----------|-------|----------
     1 |      262 |   352 | ' the'
     2 |      837 |   262 | ' ,'
     3 |      764 |   200 | ' .'
     4 |      286 |   151 | ' of'
     5 |      284 |   131 | ' to'
     6 |      290 |   108 | ' and'
     7 |      287 |    80 | ' in'
     8 |      257 |    79 | ' a'
     9 |      373 |    75 | ' was'
    10 |      569 |    59 | ' V'
    11 |    18354 |    59 | 'alky'
    12 |      220 |    59 | <SPACE>
    13 |      198 |    58 | <SPACE>
    14 |     7496 |    55 | 'ria'
    15 |      416 |    53 | ' by'
    16 |      383 |    52 | ' The'
    17 |      366 |    51 | ' "'
    18 |      796 |    50 | ' ='
    19 |      355 |    47 | ' as'
    20 |      983 |    46 | ' game'

Total unique tokens in sample: 1806

4️⃣ SAMPLE WIKITEXT CONTENT
Here are some original t

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    GPT2LMHeadModel, 
    GPT2Tokenizer, 
    TrainingArguments, 
    Trainer,
    DataCollatorForLanguageModeling,
    get_linear_schedule_with_warmup
)
from datasets import load_dataset
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, List, Tuple
import math
import json
import os

class WikiTextDataset(Dataset):
    """Improved dataset for WikiText-2 with better tokenization and filtering"""
    
    def __init__(self, texts: List[str], tokenizer, max_length: int = 1024, min_length: int = 30):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.min_length = min_length
        self.examples = []
        
        print(f"Processing {len(texts)} texts...")
        
        # Filter and clean texts
        cleaned_texts = []
        for text in texts:
            text = text.strip()
            if len(text) > self.min_length and not text.startswith('='):  # Skip headers
                # Basic cleaning
                text = text.replace('\n\n', ' ').replace('\n', ' ')
                text = ' '.join(text.split())  # Normalize whitespace
                if len(text) > self.min_length:
                    cleaned_texts.append(text)
        
        print(f"After cleaning: {len(cleaned_texts)} texts")
        
        # If no texts after cleaning, use original texts with minimal filtering
        if len(cleaned_texts) == 0:
            print("Warning: No texts after cleaning, using minimal filtering")
            for text in texts:
                text = text.strip()
                if len(text) > 10:  # Very minimal filtering
                    cleaned_texts.append(text)
        
        # Tokenize with sliding window
        for text in tqdm(cleaned_texts, desc="Tokenizing"):
            # Tokenize full text
            tokens = tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=2048)
            
            # Skip very short sequences
            if len(tokens) < 32:
                continue
            
            # Create overlapping sequences with smaller stride for better coverage
            stride = max_length // 4  # Smaller stride for more overlap
            
            for i in range(0, len(tokens) - max_length + 1, stride):
                chunk = tokens[i:i + max_length]
                if len(chunk) >= max_length:  # Ensure full length sequences
                    self.examples.append(chunk)
            
            # If the text is shorter than max_length, pad it
            if len(tokens) < max_length and len(tokens) >= 32:
                padded_tokens = tokens + [tokenizer.pad_token_id] * (max_length - len(tokens))
                self.examples.append(padded_tokens)
        
        print(f"Created {len(self.examples)} training examples")
        
        # Ensure we have at least some examples
        if len(self.examples) == 0:
            print("Warning: No examples created, creating dummy examples")
            # Create a few dummy examples as fallback
            dummy_text = "This is a sample text for training."
            tokens = tokenizer.encode(dummy_text, add_special_tokens=True)
            padded_tokens = tokens + [tokenizer.pad_token_id] * (max_length - len(tokens))
            self.examples = [padded_tokens[:max_length] for _ in range(10)]
    
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, idx):
        tokens = self.examples[idx]
        return {
            'input_ids': torch.tensor(tokens, dtype=torch.long),
            'attention_mask': torch.ones(len(tokens), dtype=torch.long)
        }

class NWPEvaluator:
    """Enhanced evaluator for Next Word Prediction metrics"""
    
    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.model.eval()
    
    def calculate_perplexity(self, dataloader) -> float:
        """Calculate perplexity with improved loss computation"""
        total_loss = 0
        total_tokens = 0
        
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Calculating perplexity"):
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                
                outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
                loss = outputs.loss
                
                # Count actual tokens (exclude padding)
                valid_tokens = attention_mask.sum().item()
                total_loss += loss.item() * valid_tokens
                total_tokens += valid_tokens
        
        avg_loss = total_loss / total_tokens
        perplexity = math.exp(avg_loss)
        return perplexity
    
    def get_next_word_probabilities(self, text: str, top_k: int = 10) -> List[Tuple[str, float]]:
        """Get top-k next word probabilities for a given text"""
        self.model.eval()
        
        # Tokenize input
        input_ids = self.tokenizer.encode(text, return_tensors='pt').to(self.device)
        
        with torch.no_grad():
            outputs = self.model(input_ids)
            logits = outputs.logits[0, -1, :]  # Get logits for next token
            
            # Apply softmax to get probabilities
            probabilities = torch.softmax(logits, dim=-1)
            
            # Get top-k tokens and their probabilities
            top_k_probs, top_k_indices = torch.topk(probabilities, top_k)
            
            # Convert to readable format
            results = []
            for i in range(top_k):
                token_id = top_k_indices[i].item()
                prob = top_k_probs[i].item()
                token_text = self.tokenizer.decode([token_id], skip_special_tokens=True)
                results.append((token_text, prob))
            
            return results
    
    def calculate_top_k_accuracy(self, dataloader, k_values: List[int] = [1, 5, 10]) -> Dict[int, float]:
        """Calculate top-k accuracy with better sampling"""
        correct_predictions = {k: 0 for k in k_values}
        total_predictions = 0
        
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Calculating top-k accuracy"):
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                
                # Process each sequence
                for seq_idx in range(min(input_ids.size(0), 4)):  # Limit for efficiency
                    sequence = input_ids[seq_idx]
                    mask = attention_mask[seq_idx]
                    
                    # Sample positions to evaluate (every 10th position)
                    valid_length = mask.sum().item()
                    positions = range(10, valid_length, 10)
                    
                    for pos in positions:
                        context = sequence[:pos].unsqueeze(0)
                        target = sequence[pos].item()
                        
                        outputs = self.model(context)
                        logits = outputs.logits[0, -1, :]
                        
                        # Get top-k predictions
                        top_k_max = max(k_values)
                        top_k_tokens = torch.topk(logits, top_k_max).indices
                        
                        # Check accuracy for different k values
                        for k in k_values:
                            if target in top_k_tokens[:k]:
                                correct_predictions[k] += 1
                        
                        total_predictions += 1
        
        # Calculate accuracies
        accuracies = {k: correct_predictions[k] / total_predictions if total_predictions > 0 else 0 
                     for k in k_values}
        return accuracies

def load_and_prepare_data():
    """Load WikiText-2 dataset with better preprocessing"""
    print("Loading WikiText-2 dataset...")
    dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
    
    # Filter and clean texts
    def clean_text(text):
        text = text.strip()
        # Skip short texts, headers, and empty lines
        if len(text) < 30 or text.startswith('=') or text.startswith('@'):  # Reduced min length
            return None
        return text
    
    train_texts = [clean_text(text) for text in dataset['train']['text']]
    valid_texts = [clean_text(text) for text in dataset['validation']['text']]
    test_texts = [clean_text(text) for text in dataset['test']['text']]
    
    # Remove None values
    train_texts = [text for text in train_texts if text is not None]
    valid_texts = [text for text in valid_texts if text is not None]
    test_texts = [text for text in test_texts if text is not None]
    
    print(f"Train texts: {len(train_texts)}")
    print(f"Validation texts: {len(valid_texts)}")
    print(f"Test texts: {len(test_texts)}")
    
    # Ensure we have enough data
    if len(train_texts) == 0:
        raise ValueError("No training texts found after filtering!")
    if len(valid_texts) == 0:
        print("Warning: No validation texts found, using subset of training data")
        valid_texts = train_texts[:min(100, len(train_texts)//10)]
    if len(test_texts) == 0:
        print("Warning: No test texts found, using subset of training data")
        test_texts = train_texts[:min(50, len(train_texts)//20)]
    
    return train_texts, valid_texts, test_texts

def setup_model_and_tokenizer():
    """Initialize GPT-2 model and tokenizer"""
    print("Setting up GPT-2 model and tokenizer...")
    model_name = "gpt2"  # Use base GPT-2 for better fine-tuning
    
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    
    # Add padding token
    tokenizer.pad_token = tokenizer.eos_token
    
    # Resize model embeddings if needed
    model.resize_token_embeddings(len(tokenizer))
    
    return model, tokenizer

def fine_tune_model(model, tokenizer, train_dataset, eval_dataset, output_dir="./gpt2-wikitext2-improved"):
    """Fine-tune GPT-2 with improved hyperparameters for better perplexity"""
    
    # Data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
        pad_to_multiple_of=8,
        return_tensors="pt"
    )
    
    # Improved training arguments for better perplexity (removed early_stopping_patience)
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=5,  # More epochs
        per_device_train_batch_size=2,  # Smaller batch size
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,  # Larger effective batch size
        warmup_steps=1000,  # More warmup
        logging_steps=50,
        save_steps=500,
        eval_steps=250,
        eval_strategy="steps",
        save_strategy="steps",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        save_total_limit=3,
        fp16=True,
        dataloader_pin_memory=True,
        learning_rate=3e-5,  # Lower learning rate
        weight_decay=0.01,
        adam_epsilon=1e-8,
        max_grad_norm=1.0,
        lr_scheduler_type="cosine",  # Cosine scheduler
        report_to=[],
        remove_unused_columns=False,
        dataloader_num_workers=4,
    )
    
    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
    )
    
    # Train the model
    print("Starting fine-tuning...")
    trainer.train()
    
    # Save the fine-tuned model
    trainer.save_model()
    tokenizer.save_pretrained(output_dir)
    
    return trainer

def evaluate_model(model, tokenizer, test_dataset, device):
    """Comprehensive evaluation of the fine-tuned model"""
    print("Evaluating fine-tuned model...")
    
    # Create test dataloader with proper collation
    def collate_fn(batch):
        input_ids = [item['input_ids'] for item in batch]
        attention_masks = [item['attention_mask'] for item in batch]
        
        # Pad sequences
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=tokenizer.pad_token_id
        )
        attention_masks = torch.nn.utils.rnn.pad_sequence(
            attention_masks, batch_first=True, padding_value=0
        )
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_masks
        }
    
    test_dataloader = DataLoader(
        test_dataset, 
        batch_size=4,  # Smaller batch size for evaluation
        shuffle=False,
        collate_fn=collate_fn
    )
    
    # Initialize evaluator
    evaluator = NWPEvaluator(model, tokenizer, device)
    
    # Calculate perplexity
    perplexity = evaluator.calculate_perplexity(test_dataloader)
    print(f"Perplexity: {perplexity:.2f}")
    
    # Calculate top-k accuracy
    accuracies = evaluator.calculate_top_k_accuracy(test_dataloader, k_values=[1, 5, 10])
    print("Top-k Accuracies:")
    for k, acc in accuracies.items():
        print(f"  Top-{k}: {acc:.4f}")
    
    return perplexity, accuracies, evaluator

def save_model_for_streamlit(model, tokenizer, output_dir="./streamlit_model"):
    """Save model and tokenizer for Streamlit app"""
    os.makedirs(output_dir, exist_ok=True)
    
    # Save model and tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    # Save model info
    model_info = {
        "model_type": "GPT2LMHeadModel",
        "vocab_size": len(tokenizer),
        "max_length": 1024,
        "pad_token_id": tokenizer.pad_token_id,
        "eos_token_id": tokenizer.eos_token_id
    }
    
    with open(os.path.join(output_dir, "model_info.json"), "w") as f:
        json.dump(model_info, f, indent=2)
    
    print(f"Model saved to {output_dir} for Streamlit app")

def main():
    """Main training pipeline with improved settings"""
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Load data
    train_texts, valid_texts, test_texts = load_and_prepare_data()
    
    # Setup model and tokenizer
    model, tokenizer = setup_model_and_tokenizer()
    model.to(device)
    
    # Create datasets with improved parameters
    print("Creating datasets...")
    # Use more data for better performance, with fallback limits
    max_train = min(5000, len(train_texts))
    max_valid = min(1000, len(valid_texts))
    max_test = min(500, len(test_texts))
    
    train_dataset = WikiTextDataset(train_texts[:max_train], tokenizer, max_length=1024)
    eval_dataset = WikiTextDataset(valid_texts[:max_valid], tokenizer, max_length=1024)
    test_dataset = WikiTextDataset(test_texts[:max_test], tokenizer, max_length=1024)
    
    print(f"Training examples: {len(train_dataset)}")
    print(f"Validation examples: {len(eval_dataset)}")
    print(f"Test examples: {len(test_dataset)}")
    
    # Final check to ensure we have valid datasets
    if len(train_dataset) == 0 or len(eval_dataset) == 0 or len(test_dataset) == 0:
        raise ValueError("One or more datasets are empty after processing!")
    
    # Ensure minimum dataset sizes
    if len(train_dataset) < 10:
        print("Warning: Very small training dataset!")
    if len(eval_dataset) < 5:
        print("Warning: Very small validation dataset!")
    
    # Fine-tune model
    trainer = fine_tune_model(model, tokenizer, train_dataset, eval_dataset)
    
    # Evaluate model
    perplexity, accuracies, evaluator = evaluate_model(model, tokenizer, test_dataset, device)
    
    # Save model for Streamlit
    save_model_for_streamlit(model, tokenizer)
    
    # Create results summary
    results = {
        'Metric': ['Perplexity', 'Top-1 Accuracy', 'Top-5 Accuracy', 'Top-10 Accuracy'],
        'Value': [f"{perplexity:.2f}", f"{accuracies[1]:.4f}", f"{accuracies[5]:.4f}", f"{accuracies[10]:.4f}"]
    }
    
    results_df = pd.DataFrame(results)
    print("\nFinal Results Summary:")
    print(results_df.to_string(index=False))
    
    # Save results
    results_df.to_csv("training_results.csv", index=False)
    
    return model, tokenizer, results_df, evaluator

if __name__ == "__main__":
    print("Improved GPT-2 Fine-tuning Pipeline for Next Word Prediction")
    print("Target Perplexity: 35-40")
    print("=" * 60)
    
    # Run the improved pipeline
    model, tokenizer, results, evaluator = main()
    
    # Test with sample predictions
    sample_texts = [
        "The president of the United States",
        "Machine learning algorithms can",
        "In the field of artificial intelligence",
        "The weather today is",
        "Scientists have discovered"
    ]
    
    print("\nSample Next Word Predictions:")
    print("=" * 50)
    
    for text in sample_texts:
        predictions = evaluator.get_next_word_probabilities(text, top_k=5)
        print(f"\nInput: '{text}'")
        print("Top 5 predictions:")
        for i, (word, prob) in enumerate(predictions, 1):
            print(f"  {i}. '{word}' (prob: {prob:.4f})")
    
    print("\nTraining complete! Model saved for Streamlit app.")

Improved GPT-2 Fine-tuning Pipeline for Next Word Prediction
Target Perplexity: 35-40
Using device: cuda
Loading WikiText-2 dataset...
Train texts: 16781
Validation texts: 1766
Test texts: 2000
Setting up GPT-2 model and tokenizer...
Creating datasets...
Processing 5000 texts...
After cleaning: 4976 texts


Tokenizing: 100%|██████████| 4976/4976 [00:05<00:00, 854.89it/s] 


Created 4366 training examples
Processing 1000 texts...
After cleaning: 999 texts


Tokenizing: 100%|██████████| 999/999 [00:01<00:00, 963.61it/s] 


Created 913 training examples
Processing 500 texts...
After cleaning: 500 texts


...

Evaluating fine-tuned model...


Calculating perplexity: 100%|██████████| 112/112 [00:39<00:00,  2.85it/s]


Perplexity: 51575.83


Calculating top-k accuracy: 100%|██████████| 112/112 [35:18<00:00, 18.92s/it]


Top-k Accuracies:
  Top-1: 0.0574
  Top-5: 0.0872
  Top-10: 0.0982
Model saved to ./streamlit_model for Streamlit app

Final Results Summary:
         Metric    Value
     Perplexity 51575.83
 Top-1 Accuracy   0.0574
 Top-5 Accuracy   0.0872
Top-10 Accuracy   0.0982

Sample Next Word Predictions:

Input: 'The president of the United States'
Top 5 predictions:
  1. ' ,' (prob: 0.0901)
  2. ' of' (prob: 0.0652)
  3. ' Conference' (prob: 0.0366)
  4. ' is' (prob: 0.0339)
  5. ' has' (prob: 0.0331)

Input: 'Machine learning algorithms can'
Top 5 predictions:
  1. ' be' (prob: 0.4474)
  2. ' also' (prob: 0.0611)
  3. ' help' (prob: 0.0222)
  4. ' take' (prob: 0.0134)
  5. ' learn' (prob: 0.0124)

Input: 'In the field of artificial intelligence'
Top 5 predictions:
  1. ' ,' (prob: 0.8455)
  2. ' (' (prob: 0.0561)
  3. ' and' (prob: 0.0111)
  4. ' research' (prob: 0.0099)
  5. ' there' (prob: 0.0087)

Input: 'The weather today is'
Top 5 predictions:
  1. ' generally' (prob: 0.0448)
  2. ' mil

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    GPT2LMHeadModel, 
    GPT2Tokenizer, 
    TrainingArguments, 
    Trainer,
    DataCollatorForLanguageModeling,
    get_linear_schedule_with_warmup
)
from datasets import load_dataset
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, List, Tuple
import math
import json
import os

class WikiTextDataset(Dataset):
    """Improved dataset for WikiText-2 with better tokenization and filtering"""
    
    def __init__(self, texts: List[str], tokenizer, max_length: int = 1024, min_length: int = 30):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.min_length = min_length
        self.examples = []
        
        print(f"Processing {len(texts)} texts...")
        
        # Filter and clean texts
        cleaned_texts = []
        for text in texts:
            text = text.strip()
            if len(text) > self.min_length and not text.startswith('='):  # Skip headers
                # Basic cleaning
                text = text.replace('\n\n', ' ').replace('\n', ' ')
                text = ' '.join(text.split())  # Normalize whitespace
                if len(text) > self.min_length:
                    cleaned_texts.append(text)
        
        print(f"After cleaning: {len(cleaned_texts)} texts")
        
        # If no texts after cleaning, use original texts with minimal filtering
        if len(cleaned_texts) == 0:
            print("Warning: No texts after cleaning, using minimal filtering")
            for text in texts:
                text = text.strip()
                if len(text) > 10:  # Very minimal filtering
                    cleaned_texts.append(text)
        
        # Tokenize with sliding window
        for text in tqdm(cleaned_texts, desc="Tokenizing"):
            # Tokenize full text
            tokens = tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=2048)
            
            # Skip very short sequences
            if len(tokens) < 16:
                continue
            
            # Create overlapping sequences with smaller stride for better coverage
            stride = max_length // 4  # Reasonable stride
            
            # If text is long enough, create sliding windows
            if len(tokens) >= max_length:
                for i in range(0, len(tokens) - max_length + 1, stride):
                    chunk = tokens[i:i + max_length]
                    self.examples.append(chunk)
            else:
                # For shorter texts, pad to max_length
                padded_tokens = tokens + [tokenizer.pad_token_id] * (max_length - len(tokens))
                self.examples.append(padded_tokens)
        
        print(f"Created {len(self.examples)} training examples")
        
        # Ensure we have at least some examples
        if len(self.examples) == 0:
            print("Warning: No examples created, creating dummy examples")
            # Create a few dummy examples as fallback
            dummy_text = "This is a sample text for training."
            tokens = tokenizer.encode(dummy_text, add_special_tokens=True)
            padded_tokens = tokens + [tokenizer.pad_token_id] * (max_length - len(tokens))
            self.examples = [padded_tokens[:max_length] for _ in range(10)]
    
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, idx):
        tokens = self.examples[idx]
        # Create proper attention mask (1 for real tokens, 0 for padding)
        attention_mask = [1 if token != self.tokenizer.pad_token_id else 0 for token in tokens]
        
        return {
            'input_ids': torch.tensor(tokens, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long)
        }

class NWPEvaluator:
    """Enhanced evaluator for Next Word Prediction metrics"""
    
    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.model.eval()
    
    def calculate_perplexity(self, dataloader) -> float:
        """Calculate perplexity with improved loss computation"""
        total_loss = 0
        total_tokens = 0
        
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Calculating perplexity"):
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                
                outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
                loss = outputs.loss
                
                # Count actual tokens (exclude padding)
                valid_tokens = attention_mask.sum().item()
                total_loss += loss.item() * valid_tokens
                total_tokens += valid_tokens
        
        avg_loss = total_loss / total_tokens
        perplexity = math.exp(avg_loss)
        return perplexity
    
    def get_next_word_probabilities(self, text: str, top_k: int = 10) -> List[Tuple[str, float]]:
        """Get top-k next word probabilities for a given text"""
        self.model.eval()
        
        # Tokenize input
        input_ids = self.tokenizer.encode(text, return_tensors='pt').to(self.device)
        
        with torch.no_grad():
            outputs = self.model(input_ids)
            logits = outputs.logits[0, -1, :]  # Get logits for next token
            
            # Apply softmax to get probabilities
            probabilities = torch.softmax(logits, dim=-1)
            
            # Get top-k tokens and their probabilities
            top_k_probs, top_k_indices = torch.topk(probabilities, top_k)
            
            # Convert to readable format
            results = []
            for i in range(top_k):
                token_id = top_k_indices[i].item()
                prob = top_k_probs[i].item()
                token_text = self.tokenizer.decode([token_id], skip_special_tokens=True)
                results.append((token_text, prob))
            
            return results
    
    def calculate_top_k_accuracy(self, dataloader, k_values: List[int] = [1, 5, 10]) -> Dict[int, float]:
        """Calculate top-k accuracy with better sampling"""
        correct_predictions = {k: 0 for k in k_values}
        total_predictions = 0
        
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Calculating top-k accuracy"):
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                
                # Process each sequence
                for seq_idx in range(min(input_ids.size(0), 4)):  # Limit for efficiency
                    sequence = input_ids[seq_idx]
                    mask = attention_mask[seq_idx]
                    
                    # Sample positions to evaluate (every 10th position)
                    valid_length = mask.sum().item()
                    positions = range(10, valid_length, 10)
                    
                    for pos in positions:
                        context = sequence[:pos].unsqueeze(0)
                        target = sequence[pos].item()
                        
                        outputs = self.model(context)
                        logits = outputs.logits[0, -1, :]
                        
                        # Get top-k predictions
                        top_k_max = max(k_values)
                        top_k_tokens = torch.topk(logits, top_k_max).indices
                        
                        # Check accuracy for different k values
                        for k in k_values:
                            if target in top_k_tokens[:k]:
                                correct_predictions[k] += 1
                        
                        total_predictions += 1
        
        # Calculate accuracies
        accuracies = {k: correct_predictions[k] / total_predictions if total_predictions > 0 else 0 
                     for k in k_values}
        return accuracies

def load_and_prepare_data():
    """Load WikiText-2 dataset with better preprocessing"""
    print("Loading WikiText-2 dataset...")
    dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
    
    # Filter and clean texts with less aggressive filtering
    def clean_text(text):
        text = text.strip()
        # More lenient filtering - only skip very short texts and obvious headers
        if len(text) < 20 or text.startswith('= =') or text.startswith(' = '):
            return None
        return text
    
    train_texts = [clean_text(text) for text in dataset['train']['text']]
    valid_texts = [clean_text(text) for text in dataset['validation']['text']]
    test_texts = [clean_text(text) for text in dataset['test']['text']]
    
    # Remove None values
    train_texts = [text for text in train_texts if text is not None]
    valid_texts = [text for text in valid_texts if text is not None]
    test_texts = [text for text in test_texts if text is not None]
    
    print(f"Train texts: {len(train_texts)}")
    print(f"Validation texts: {len(valid_texts)}")
    print(f"Test texts: {len(test_texts)}")
    
    # Ensure we have enough data
    if len(train_texts) == 0:
        raise ValueError("No training texts found after filtering!")
    if len(valid_texts) == 0:
        print("Warning: No validation texts found, using subset of training data")
        valid_texts = train_texts[:min(100, len(train_texts)//10)]
    if len(test_texts) == 0:
        print("Warning: No test texts found, using subset of training data")
        test_texts = train_texts[:min(50, len(train_texts)//20)]
    
    return train_texts, valid_texts, test_texts

def setup_model_and_tokenizer():
    """Initialize GPT-2 model and tokenizer"""
    print("Setting up GPT-2 model and tokenizer...")
    model_name = "gpt2"  # Use base GPT-2 for better fine-tuning
    
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    
    # Add padding token
    tokenizer.pad_token = tokenizer.eos_token
    
    # Resize model embeddings if needed
    model.resize_token_embeddings(len(tokenizer))
    
    return model, tokenizer

def fine_tune_model(model, tokenizer, train_dataset, eval_dataset, output_dir="./gpt2-wikitext2-improved"):
    """Fine-tune GPT-2 with improved hyperparameters for better perplexity"""
    
    # Data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
        pad_to_multiple_of=8,
        return_tensors="pt"
    )
    
    # Improved training arguments for better perplexity (fixed parameters)
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=3,  # Reduced epochs for small dataset
        per_device_train_batch_size=4,  # Larger batch size
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=2,  # Smaller accumulation
        warmup_steps=500,  # Reduced warmup for small dataset
        logging_steps=50,
        save_steps=250,
        eval_steps=125,
        eval_strategy="steps",
        save_strategy="steps",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        save_total_limit=2,
        fp16=True,
        dataloader_pin_memory=True,
        learning_rate=5e-5,  # More conservative learning rate
        weight_decay=0.01,
        adam_epsilon=1e-8,
        max_grad_norm=1.0,
        lr_scheduler_type="linear",  # Linear scheduler for stability
        report_to=[],
        remove_unused_columns=False,
        dataloader_num_workers=2,
        push_to_hub=False,
        hub_model_id=None,
    )
    
    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
    )
    
    # Train the model
    print("Starting fine-tuning...")
    trainer.train()
    
    # Save the fine-tuned model
    trainer.save_model()
    tokenizer.save_pretrained(output_dir)
    
    return trainer

def evaluate_model(model, tokenizer, test_dataset, device):
    """Comprehensive evaluation of the fine-tuned model"""
    print("Evaluating fine-tuned model...")
    
    # Create test dataloader with proper collation
    def collate_fn(batch):
        input_ids = [item['input_ids'] for item in batch]
        attention_masks = [item['attention_mask'] for item in batch]
        
        # Pad sequences
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=tokenizer.pad_token_id
        )
        attention_masks = torch.nn.utils.rnn.pad_sequence(
            attention_masks, batch_first=True, padding_value=0
        )
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_masks
        }
    
    test_dataloader = DataLoader(
        test_dataset, 
        batch_size=4,  # Smaller batch size for evaluation
        shuffle=False,
        collate_fn=collate_fn
    )
    
    # Initialize evaluator
    evaluator = NWPEvaluator(model, tokenizer, device)
    
    # Calculate perplexity
    perplexity = evaluator.calculate_perplexity(test_dataloader)
    print(f"Perplexity: {perplexity:.2f}")
    
    # Calculate top-k accuracy
    accuracies = evaluator.calculate_top_k_accuracy(test_dataloader, k_values=[1, 5, 10])
    print("Top-k Accuracies:")
    for k, acc in accuracies.items():
        print(f"  Top-{k}: {acc:.4f}")
    
    return perplexity, accuracies, evaluator

def save_model_for_streamlit(model, tokenizer, output_dir="./streamlit_model"):
    """Save model and tokenizer for Streamlit app"""
    os.makedirs(output_dir, exist_ok=True)
    
    # Save model and tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    # Save model info
    model_info = {
        "model_type": "GPT2LMHeadModel",
        "vocab_size": len(tokenizer),
        "max_length": 1024,
        "pad_token_id": tokenizer.pad_token_id,
        "eos_token_id": tokenizer.eos_token_id
    }
    
    with open(os.path.join(output_dir, "model_info.json"), "w") as f:
        json.dump(model_info, f, indent=2)
    
    print(f"Model saved to {output_dir} for Streamlit app")

def main():
    """Main training pipeline with improved settings"""
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Load data
    train_texts, valid_texts, test_texts = load_and_prepare_data()
    
    # Setup model and tokenizer
    model, tokenizer = setup_model_and_tokenizer()
    model.to(device)
    
    # Create datasets with improved parameters
    print("Creating datasets...")
    # Use ALL available data to maximize training examples
    train_dataset = WikiTextDataset(train_texts, tokenizer, max_length=512)  # Shorter sequences
    eval_dataset = WikiTextDataset(valid_texts, tokenizer, max_length=512)
    test_dataset = WikiTextDataset(test_texts, tokenizer, max_length=512)
    
    print(f"Training examples: {len(train_dataset)}")
    print(f"Validation examples: {len(eval_dataset)}")
    print(f"Test examples: {len(test_dataset)}")
    
    # Final check to ensure we have valid datasets
    if len(train_dataset) == 0 or len(eval_dataset) == 0 or len(test_dataset) == 0:
        raise ValueError("One or more datasets are empty after processing!")
    
    # Ensure minimum dataset sizes
    if len(train_dataset) < 100:
        print("Warning: Very small training dataset! Consider using more data or less aggressive filtering.")
    if len(eval_dataset) < 10:
        print("Warning: Very small validation dataset!")
    
    # Fine-tune model
    trainer = fine_tune_model(model, tokenizer, train_dataset, eval_dataset)
    
    # Evaluate model
    perplexity, accuracies, evaluator = evaluate_model(model, tokenizer, test_dataset, device)
    
    # Save model for Streamlit
    save_model_for_streamlit(model, tokenizer)
    
    # Create results summary
    results = {
        'Metric': ['Perplexity', 'Top-1 Accuracy', 'Top-5 Accuracy', 'Top-10 Accuracy'],
        'Value': [f"{perplexity:.2f}", f"{accuracies[1]:.4f}", f"{accuracies[5]:.4f}", f"{accuracies[10]:.4f}"]
    }
    
    results_df = pd.DataFrame(results)
    print("\nFinal Results Summary:")
    print(results_df.to_string(index=False))
    
    # Save results
    results_df.to_csv("training_results.csv", index=False)
    
    return model, tokenizer, results_df, evaluator

if __name__ == "__main__":
    print("Improved GPT-2 Fine-tuning Pipeline for Next Word Prediction")
    print("Target Perplexity: 35-40")
    print("=" * 60)
    
    # Run the improved pipeline
    model, tokenizer, results, evaluator = main()
    
    # Test with sample predictions
    sample_texts = [
        "The president of the United States",
        "Machine learning algorithms can",
        "In the field of artificial intelligence",
        "The weather today is",
        "Scientists have discovered"
    ]
    
    print("\nSample Next Word Predictions:")
    print("=" * 50)
    
    for text in sample_texts:
        predictions = evaluator.get_next_word_probabilities(text, top_k=5)
        print(f"\nInput: '{text}'")
        print("Top 5 predictions:")
        for i, (word, prob) in enumerate(predictions, 1):
            print(f"  {i}. '{word}' (prob: {prob:.4f})")
    
    print("\nTraining complete! Model saved for Streamlit app.")

Improved GPT-2 Fine-tuning Pipeline for Next Word Prediction
Target Perplexity: 35-40
Using device: cuda
Loading WikiText-2 dataset...
Train texts: 17537
Validation texts: 1850
Test texts: 2114
Setting up GPT-2 model and tokenizer...
Creating datasets...
Processing 17537 texts...
After cleaning: 16736 texts


Tokenizing: 100%|██████████| 16736/16736 [00:18<00:00, 900.52it/s] 


Created 15868 training examples
Processing 1850 texts...
After cleaning: 1762 texts


Tokenizing: 100%|██████████| 1762/1762 [00:01<00:00, 1027.72it/s]


Created 1686 training examples
Processing 2114 texts...
After cleaning: 1995 texts


...

Evaluating fine-tuned model...


Calculating perplexity: 100%|██████████| 473/473 [01:28<00:00,  5.37it/s]


Perplexity: 4275.16


Calculating top-k accuracy: 100%|██████████| 473/473 [05:41<00:00,  1.39it/s]


Top-k Accuracies:
  Top-1: 0.3996
  Top-5: 0.6193
  Top-10: 0.6925
Model saved to ./streamlit_model for Streamlit app

Final Results Summary:
         Metric   Value
     Perplexity 4275.16
 Top-1 Accuracy  0.3996
 Top-5 Accuracy  0.6193
Top-10 Accuracy  0.6925

Sample Next Word Predictions:

Input: 'The president of the United States'
Top 5 predictions:
  1. ' ,' (prob: 0.1766)
  2. ' was' (prob: 0.0511)
  3. ' is' (prob: 0.0421)
  4. ' of' (prob: 0.0377)
  5. ' has' (prob: 0.0341)

Input: 'Machine learning algorithms can'
Top 5 predictions:
  1. ' be' (prob: 0.4724)
  2. ' also' (prob: 0.0639)
  3. ' perform' (prob: 0.0197)
  4. ' learn' (prob: 0.0191)
  5. ' help' (prob: 0.0172)

Input: 'In the field of artificial intelligence'
Top 5 predictions:
  1. ' ,' (prob: 0.8406)
  2. ' (' (prob: 0.0638)
  3. ' and' (prob: 0.0150)
  4. ' research' (prob: 0.0105)
  5. ' there' (prob: 0.0084)

Input: 'The weather today is'
Top 5 predictions:
  1. ' generally' (prob: 0.1253)
  2. ' mild' (prob:

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    GPT2LMHeadModel, 
    GPT2Tokenizer, 
    TrainingArguments, 
    Trainer,
    DataCollatorForLanguageModeling,
    get_linear_schedule_with_warmup
)
from datasets import load_dataset
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, List, Tuple
import math
import json
import os
import warnings

# Suppress common warnings
warnings.filterwarnings("ignore", message="Was asked to gather along dimension 0")

class WikiTextDataset(Dataset):
    """Fixed dataset with proper tokenization and no sliding window issues"""
    
    def __init__(self, texts: List[str], tokenizer, max_length: int = 512, min_length: int = 50):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.examples = []
        
        print(f"Processing {len(texts)} texts...")
        
        # More conservative text filtering
        valid_texts = []
        for text in texts:
            text = text.strip()
            # Skip empty texts, headers, and very short texts
            if (len(text) > min_length and 
                not text.startswith('=') and 
                not text.startswith(' =') and
                len(text.split()) > 10):  # At least 10 words
                # Basic cleaning
                text = ' '.join(text.split())  # Normalize whitespace
                valid_texts.append(text)
        
        print(f"Valid texts after filtering: {len(valid_texts)}")
        
        # Concatenate texts with separator instead of sliding window
        if valid_texts:
            # Join texts with EOS token
            combined_text = f" {tokenizer.eos_token} ".join(valid_texts)
            
            # Tokenize the combined text
            tokens = tokenizer.encode(combined_text, add_special_tokens=False)
            
            # Split into chunks of max_length
            for i in range(0, len(tokens), max_length):
                chunk = tokens[i:i + max_length]
                if len(chunk) >= 32:  # Minimum chunk size
                    # Pad if necessary
                    if len(chunk) < max_length:
                        chunk = chunk + [tokenizer.pad_token_id] * (max_length - len(chunk))
                    self.examples.append(chunk)
        
        print(f"Created {len(self.examples)} training examples")
        
        # Fallback if no examples created
        if len(self.examples) == 0:
            print("ERROR: No examples created! Using fallback...")
            # Create meaningful fallback examples
            fallback_texts = [
                "This is a sample text for language modeling training.",
                "Natural language processing is a field of artificial intelligence.",
                "Machine learning models learn patterns from data.",
                "Deep learning uses neural networks with multiple layers.",
                "The quick brown fox jumps over the lazy dog.",
                "Artificial intelligence has revolutionized many industries.",
                "Computer vision enables machines to understand images.",
                "Speech recognition converts audio to text automatically.",
                "Neural networks are inspired by biological brain structures.",
                "Data science combines statistics with programming skills."
            ]
            
            for text in fallback_texts:
                tokens = tokenizer.encode(text, add_special_tokens=True)
                if len(tokens) < max_length:
                    tokens = tokens + [tokenizer.pad_token_id] * (max_length - len(tokens))
                else:
                    tokens = tokens[:max_length]
                self.examples.append(tokens)
    
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, idx):
        tokens = self.examples[idx]
        # Ensure we have proper tensors
        input_ids = torch.tensor(tokens, dtype=torch.long)
        
        # Create attention mask (1 for real tokens, 0 for padding)
        attention_mask = torch.tensor(
            [1 if token != self.tokenizer.pad_token_id else 0 for token in tokens], 
            dtype=torch.long
        )
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': input_ids.clone()  # For language modeling, labels = input_ids
        }


class NWPEvaluator:
    """Fixed evaluator with proper perplexity calculation"""
    
    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.model.eval()
    
    def calculate_perplexity(self, dataloader) -> float:
        """Fixed perplexity calculation that properly handles padding"""
        total_loss = 0
        total_tokens = 0
        
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Calculating perplexity"):
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                # Forward pass
                outputs = self.model(
                    input_ids=input_ids, 
                    attention_mask=attention_mask, 
                    labels=labels
                )
                
                # Get loss and logits
                loss = outputs.loss
                logits = outputs.logits
                
                # Calculate loss manually to ensure proper handling
                shift_logits = logits[..., :-1, :].contiguous()
                shift_labels = labels[..., 1:].contiguous()
                shift_attention_mask = attention_mask[..., 1:].contiguous()
                
                # Flatten for loss calculation
                shift_logits = shift_logits.view(-1, shift_logits.size(-1))
                shift_labels = shift_labels.view(-1)
                shift_attention_mask = shift_attention_mask.view(-1)
                
                # Calculate loss only for non-padded tokens
                loss_fct = nn.CrossEntropyLoss(reduction='none')
                losses = loss_fct(shift_logits, shift_labels)
                
                # Mask out padded tokens
                masked_losses = losses * shift_attention_mask.float()
                
                # Sum up losses and token counts
                total_loss += masked_losses.sum().item()
                total_tokens += shift_attention_mask.sum().item()
        
        if total_tokens == 0:
            return float('inf')
        
        avg_loss = total_loss / total_tokens
        perplexity = math.exp(avg_loss)
        
        # Cap perplexity at reasonable value to detect issues
        if perplexity > 1000:
            print(f"WARNING: Very high perplexity {perplexity:.2f} detected!")
            print(f"Average loss: {avg_loss:.4f}")
            print(f"Total tokens: {total_tokens}")
        
        return perplexity
    
    def get_next_word_probabilities(self, text: str, top_k: int = 10) -> List[Tuple[str, float]]:
        """Get top-k next word probabilities for a given text"""
        self.model.eval()
        
        # Tokenize input
        input_ids = self.tokenizer.encode(text, return_tensors='pt').to(self.device)
        
        with torch.no_grad():
            outputs = self.model(input_ids)
            logits = outputs.logits[0, -1, :]  # Get logits for next token
            
            # Apply softmax to get probabilities
            probabilities = torch.softmax(logits, dim=-1)
            
            # Get top-k tokens and their probabilities
            top_k_probs, top_k_indices = torch.topk(probabilities, top_k)
            
            # Convert to readable format
            results = []
            for i in range(top_k):
                token_id = top_k_indices[i].item()
                prob = top_k_probs[i].item()
                token_text = self.tokenizer.decode([token_id], skip_special_tokens=True)
                results.append((token_text, prob))
            
            return results
    
    def calculate_top_k_accuracy(self, dataloader, k_values: List[int] = [1, 5, 10]) -> Dict[int, float]:
        """Calculate top-k accuracy with better sampling"""
        correct_predictions = {k: 0 for k in k_values}
        total_predictions = 0
        
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Calculating top-k accuracy"):
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                
                # Process each sequence
                for seq_idx in range(min(input_ids.size(0), 2)):  # Limit for efficiency
                    sequence = input_ids[seq_idx]
                    mask = attention_mask[seq_idx]
                    
                    # Sample positions to evaluate (every 20th position)
                    valid_length = mask.sum().item()
                    positions = range(20, valid_length - 1, 20)
                    
                    for pos in positions:
                        if pos >= valid_length - 1:
                            continue
                            
                        context = sequence[:pos].unsqueeze(0)
                        target = sequence[pos].item()
                        
                        outputs = self.model(context)
                        logits = outputs.logits[0, -1, :]
                        
                        # Get top-k predictions
                        top_k_max = max(k_values)
                        top_k_tokens = torch.topk(logits, top_k_max).indices
                        
                        # Check accuracy for different k values
                        for k in k_values:
                            if target in top_k_tokens[:k]:
                                correct_predictions[k] += 1
                        
                        total_predictions += 1
        
        # Calculate accuracies
        accuracies = {k: correct_predictions[k] / total_predictions if total_predictions > 0 else 0 
                     for k in k_values}
        return accuracies


def debug_dataset(dataset, tokenizer, num_samples=3):
    """Debug function to examine dataset contents"""
    print(f"\nDEBUG: Dataset has {len(dataset)} examples")
    
    for i in range(min(num_samples, len(dataset))):
        example = dataset[i]
        input_ids = example['input_ids']
        attention_mask = example['attention_mask']
        
        print(f"\nExample {i+1}:")
        print(f"Input IDs shape: {input_ids.shape}")
        print(f"Attention mask shape: {attention_mask.shape}")
        print(f"Non-padding tokens: {attention_mask.sum().item()}")
        
        # Decode first 50 tokens
        decoded = tokenizer.decode(input_ids[:50], skip_special_tokens=False)
        print(f"First 50 tokens: {decoded}")
        
        # Check for issues
        if attention_mask.sum().item() < 10:
            print("WARNING: Very few non-padding tokens!")
        if input_ids.max().item() >= len(tokenizer):
            print("WARNING: Token ID out of vocabulary range!")


def load_and_prepare_data():
    """Load WikiText-2 dataset with better preprocessing"""
    print("Loading WikiText-2 dataset...")
    try:
        dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
    except Exception as e:
        print(f"Error loading dataset: {e}")
        print("Using fallback data...")
        # Create fallback dataset
        fallback_train = [
            "Machine learning is a subset of artificial intelligence that focuses on algorithms.",
            "Deep learning uses neural networks with multiple layers to process data.",
            "Natural language processing enables computers to understand human language.",
            "Computer vision allows machines to interpret and analyze visual information.",
            "Data science combines statistics, programming, and domain expertise.",
            "Artificial intelligence has applications in healthcare, finance, and transportation.",
            "Neural networks are inspired by the structure of biological brains.",
            "Big data refers to large volumes of structured and unstructured data.",
            "Cloud computing provides scalable and flexible IT resources over the internet.",
            "Cybersecurity protects digital systems from threats and attacks."
        ] * 20  # Repeat to create more training data
        
        fallback_valid = fallback_train[:50]
        fallback_test = fallback_train[:25]
        
        return fallback_train, fallback_valid, fallback_test
    
    # Filter and clean texts with less aggressive filtering
    def clean_text(text):
        text = text.strip()
        # More lenient filtering - only skip very short texts and obvious headers
        if len(text) < 30 or text.startswith('= =') or text.startswith(' = '):
            return None
        return text
    
    train_texts = [clean_text(text) for text in dataset['train']['text']]
    valid_texts = [clean_text(text) for text in dataset['validation']['text']]
    test_texts = [clean_text(text) for text in dataset['test']['text']]
    
    # Remove None values
    train_texts = [text for text in train_texts if text is not None]
    valid_texts = [text for text in valid_texts if text is not None]
    test_texts = [text for text in test_texts if text is not None]
    
    print(f"Train texts: {len(train_texts)}")
    print(f"Validation texts: {len(valid_texts)}")
    print(f"Test texts: {len(test_texts)}")
    
    # Ensure we have enough data
    if len(train_texts) == 0:
        print("Warning: No training texts found, using fallback")
        train_texts = ["This is sample training text for language modeling."] * 100
    if len(valid_texts) == 0:
        print("Warning: No validation texts found, using subset of training data")
        valid_texts = train_texts[:min(50, len(train_texts)//10)]
    if len(test_texts) == 0:
        print("Warning: No test texts found, using subset of training data")
        test_texts = train_texts[:min(25, len(train_texts)//20)]
    
    return train_texts, valid_texts, test_texts


def setup_model_and_tokenizer():
    """Initialize GPT-2 model and tokenizer"""
    print("Setting up GPT-2 model and tokenizer...")
    model_name = "gpt2"  # Use base GPT-2 for better fine-tuning
    
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    
    # Add padding token
    tokenizer.pad_token = tokenizer.eos_token
    
    # Resize model embeddings if needed
    model.resize_token_embeddings(len(tokenizer))
    
    return model, tokenizer


class CustomDataCollator:
    """Custom data collator to ensure proper batch formatting"""
    
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    
    def __call__(self, features):
        # Extract components
        input_ids = [f['input_ids'] for f in features]
        attention_masks = [f['attention_mask'] for f in features]
        labels = [f['labels'] for f in features]
        
        # Stack tensors
        batch = {
            'input_ids': torch.stack(input_ids),
            'attention_mask': torch.stack(attention_masks),
            'labels': torch.stack(labels)
        }
        
        return batch


def fine_tune_model(model, tokenizer, train_dataset, eval_dataset, output_dir="./gpt2-wikitext2-fixed"):
    """Fine-tune GPT-2 with improved hyperparameters"""
    
    # Custom data collator
    data_collator = CustomDataCollator(tokenizer)
    
    # Conservative training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=3,  # Moderate number of epochs
        per_device_train_batch_size=2,  # Small batch size
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,  # Increased accumulation
        warmup_steps=100,  # Reduced warmup
        logging_steps=25,
        save_steps=500,
        eval_steps=250,
        eval_strategy="steps",
        save_strategy="steps",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        save_total_limit=2,
        fp16=False,  # Disable fp16 for stability
        dataloader_pin_memory=True,
        learning_rate=1e-5,  # Very conservative learning rate
        weight_decay=0.01,
        adam_epsilon=1e-8,
        max_grad_norm=1.0,
        lr_scheduler_type="cosine",
        report_to=[],
        remove_unused_columns=False,
        dataloader_num_workers=0,  # Disable multiprocessing for debugging
        push_to_hub=False,
        hub_model_id=None,
        logging_dir="./logs",
        logging_strategy="steps",
        logging_first_step=True,
        prediction_loss_only=True,
    )
    
    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
    )
    
    # Train the model
    print("Starting fine-tuning...")
    trainer.train()
    
    # Save the fine-tuned model
    trainer.save_model()
    tokenizer.save_pretrained(output_dir)
    
    return trainer


def evaluate_model(model, tokenizer, test_dataset, device):
    """Comprehensive evaluation of the fine-tuned model"""
    print("Evaluating fine-tuned model...")
    
    # Create test dataloader
    data_collator = CustomDataCollator(tokenizer)
    test_dataloader = DataLoader(
        test_dataset, 
        batch_size=2,  # Small batch size for evaluation
        shuffle=False,
        collate_fn=data_collator
    )
    
    # Initialize evaluator
    evaluator = NWPEvaluator(model, tokenizer, device)
    
    # Calculate perplexity
    perplexity = evaluator.calculate_perplexity(test_dataloader)
    print(f"Perplexity: {perplexity:.2f}")
    
    # Calculate top-k accuracy
    accuracies = evaluator.calculate_top_k_accuracy(test_dataloader, k_values=[1, 5, 10])
    print("Top-k Accuracies:")
    for k, acc in accuracies.items():
        print(f"  Top-{k}: {acc:.4f}")
    
    return perplexity, accuracies, evaluator


def save_model_for_streamlit(model, tokenizer, output_dir="./streamlit_model"):
    """Save model and tokenizer for Streamlit app"""
    os.makedirs(output_dir, exist_ok=True)
    
    # Save model and tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    # Save model info
    model_info = {
        "model_type": "GPT2LMHeadModel",
        "vocab_size": len(tokenizer),
        "max_length": 512,
        "pad_token_id": tokenizer.pad_token_id,
        "eos_token_id": tokenizer.eos_token_id
    }
    
    with open(os.path.join(output_dir, "model_info.json"), "w") as f:
        json.dump(model_info, f, indent=2)
    
    print(f"Model saved to {output_dir} for Streamlit app")


def main():
    """Main training pipeline with comprehensive fixes"""
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Load data
    train_texts, valid_texts, test_texts = load_and_prepare_data()
    
    # Debug raw data
    print("\nSample training texts:")
    for i, text in enumerate(train_texts[:3]):
        print(f"{i+1}: {text[:200]}...")
    
    # Setup model and tokenizer
    model, tokenizer = setup_model_and_tokenizer()
    model.to(device)
    
    # Create datasets
    print("Creating datasets...")
    train_dataset = WikiTextDataset(train_texts, tokenizer, max_length=512)
    eval_dataset = WikiTextDataset(valid_texts, tokenizer, max_length=512)
    test_dataset = WikiTextDataset(test_texts, tokenizer, max_length=512)
    
    print(f"Training examples: {len(train_dataset)}")
    print(f"Validation examples: {len(eval_dataset)}")
    print(f"Test examples: {len(test_dataset)}")
    
    # Debug datasets
    debug_dataset(train_dataset, tokenizer, num_samples=2)
    debug_dataset(eval_dataset, tokenizer, num_samples=1)
    
    # Ensure we have valid datasets
    if len(train_dataset) == 0 or len(eval_dataset) == 0 or len(test_dataset) == 0:
        raise ValueError("One or more datasets are empty after processing!")
    
    # Fine-tune model
    trainer = fine_tune_model(model, tokenizer, train_dataset, eval_dataset)
    
    # Evaluate model
    perplexity, accuracies, evaluator = evaluate_model(model, tokenizer, test_dataset, device)
    
    # Save model for Streamlit
    save_model_for_streamlit(model, tokenizer)
    
    # Create results summary
    results = {
        'Metric': ['Perplexity', 'Top-1 Accuracy', 'Top-5 Accuracy', 'Top-10 Accuracy'],
        'Value': [f"{perplexity:.2f}", f"{accuracies[1]:.4f}", f"{accuracies[5]:.4f}", f"{accuracies[10]:.4f}"]
    }
    
    results_df = pd.DataFrame(results)
    print("\nFinal Results Summary:")
    print(results_df.to_string(index=False))
    
    # Save results
    results_df.to_csv("training_results.csv", index=False)
    
    return model, tokenizer, results_df, evaluator


if __name__ == "__main__":
    print("Fixed GPT-2 Fine-tuning Pipeline for Next Word Prediction")
    print("Target Perplexity: 35-40")
    print("=" * 60)
    
    try:
        # Run the fixed pipeline
        model, tokenizer, results, evaluator = main()
        
        # Test with sample predictions
        sample_texts = [
            "The president of the United States",
            "Machine learning algorithms can",
            "In the field of artificial intelligence",
            "The weather today is",
            "Scientists have discovered"
        ]
        
        print("\nSample Next Word Predictions:")
        print("=" * 50)
        
        for text in sample_texts:
            try:
                predictions = evaluator.get_next_word_probabilities(text, top_k=5)
                print(f"\nInput: '{text}'")
                print("Top 5 predictions:")
                for i, (word, prob) in enumerate(predictions, 1):
                    print(f"  {i}. '{word}' (prob: {prob:.4f})")
            except Exception as e:
                print(f"Error with text '{text}': {e}")
        
        print("\nTraining complete! Model saved for Streamlit app.")
        
    except Exception as e:
        print(f"Error during training: {e}")
        import traceback
        traceback.print_exc()

Fixed GPT-2 Fine-tuning Pipeline for Next Word Prediction
Target Perplexity: 35-40
Using device: cuda
Loading WikiText-2 dataset...
Train texts: 16965
Validation texts: 1784
Test texts: 2017

Sample training texts:
1: Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ pl...
2: The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjus...
3: It met with positive sales in Japan , and was praised by both Japanese and western critics . After release , it received downloadable content , along with an expanded edition in November of that year ...
Setting up GPT-2 model and tokenizer...
Creating datasets...
Processing 16965 texts...
Valid texts after filtering: 15942


Token indices sequence length is longer than the specified maximum sequence length for this model (2314275 > 1024). Running this sequence through the model will result in indexing errors


Created 4521 training examples
Processing 1784 texts...
Valid texts after filtering: 1708
Created 469 training examples
Processing 2017 texts...
Valid texts after filtering: 1907
Created 535 training examples
Training examples: 4521
Validation examples: 469
Test examples: 535

DEBUG: Dataset has 4521 examples

Example 1:
Input IDs shape: torch.Size([512])
Attention mask shape: torch.Size([512])
Non-padding tokens: 509
First 50 tokens: Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III

Example 2:
Input IDs shape: torch.Size([512])
Attention mask shape: torch.Size([512])
Non-padding tokens: 510
First 50 tokens:  is sealed off to the player . Outside missions , the player characters rest in a camp , where units can be customized and character growth occurs . Alongside the main story missions are character @-@ specific sub missions relating to different squad members . After the

DE

/tmp/ipykernel_36/1794405346.py:424: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
250,1.728600,3.304678
500,1.691300,3.263170
750,1.690500,3.254225


...

Evaluating fine-tuned model...


Calculating perplexity: 100%|██████████| 268/268 [00:26<00:00, 10.17it/s]


Perplexity: 26.33


Calculating top-k accuracy: 100%|██████████| 268/268 [05:36<00:00,  1.26s/it]


Top-k Accuracies:
  Top-1: 0.3890
  Top-5: 0.6075
  Top-10: 0.6800
Model saved to ./streamlit_model for Streamlit app

Final Results Summary:
         Metric  Value
     Perplexity  26.33
 Top-1 Accuracy 0.3890
 Top-5 Accuracy 0.6075
Top-10 Accuracy 0.6800

Sample Next Word Predictions:

Input: 'The president of the United States'
Top 5 predictions:
  1. ',' (prob: 0.0552)
  2. ' has' (prob: 0.0548)
  3. ' is' (prob: 0.0418)
  4. ' and' (prob: 0.0370)
  5. ' ,' (prob: 0.0308)

Input: 'Machine learning algorithms can'
Top 5 predictions:
  1. ' be' (prob: 0.3429)
  2. ' also' (prob: 0.0418)
  3. ' help' (prob: 0.0330)
  4. ' learn' (prob: 0.0269)
  5. ' now' (prob: 0.0164)

Input: 'In the field of artificial intelligence'
Top 5 predictions:
  1. ' ,' (prob: 0.5697)
  2. ',' (prob: 0.1071)
  3. ' (' (prob: 0.0935)
  4. ' and' (prob: 0.0258)
  5. ' research' (prob: 0.0156)

Input: 'The weather today is'
Top 5 predictions:
  1. ' very' (prob: 0.0423)
  2. ' a' (prob: 0.0297)
  3. ' not' (pr

In [10]:
import gradio as gr
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch.nn.functional as F

# Load your fine-tuned GPT-2 model
model_path = "./streamlit_model"  # your folder
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
model.eval()

def next_word_prediction(prompt, top_k=5, temperature=1.0):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    with torch.no_grad():
        logits = model(input_ids).logits
        next_token_logits = logits[0, -1, :] / temperature
        probs = torch.nn.functional.softmax(next_token_logits, dim=-1)
        top_probs, top_indices = torch.topk(probs, int(top_k))  # ← Fixed this line
        predictions = []
        for idx, prob in zip(top_indices, top_probs):
            token = tokenizer.decode([idx.item()]).strip()
            if token:
                predictions.append(f"{token} ({prob.item()*100:.2f}%)")
        return "\n".join(predictions)


# Gradio UI
gr.Interface(
    fn=next_word_prediction,
    inputs=[
        gr.Textbox(label="Enter a prompt"),
        gr.Slider(1, 10, value=5, label="Top-K"),
        gr.Slider(0.5, 2.0, value=1.0, label="Temperature")
    ],
    outputs=gr.Textbox(label="Top Predictions"),
    title="🧠 Next Word Prediction with Fine-Tuned GPT-2"
).launch(share=True)


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://8f5cb32a8354c417dc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
